See eventual readme and explanation page for full comments and motivation on this project.

#### import statements

In [345]:
from trueskill import Rating, quality_1vs1, rate_1vs1, rate, TrueSkill
import trueskill

import pandas as pd
import numpy as np

import numpy.linalg as la
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import json
import requests
import re
import os
import itertools
import bs4
import networkx
import urllib.request
from itertools import combinations
from bs4 import BeautifulSoup
import collections
from collections import Counter

import time

import warnings
warnings.filterwarnings(action='ignore')

from functools import reduce

from datetime import date
from dateutil.relativedelta import relativedelta

In [346]:
#setting today's date for export statements

today = date.today()
today = str(today.strftime("%Y")) + str(today.strftime("%m")) + str(today.strftime("%d"))
today

'20210509'

#### custom functions for parsing scraped strings

In [347]:
def match_parser(item):
    '''
    categorizes matches by number of participants and type for parsing and strips text from string
    '''
    matchguide = pd.read_csv('aew_matchguide.csv') #document i made of match types to strip that out from the text
    for i, j in matchguide.iterrows():
        if item[:j.len] == j.specific:
            item = item[j.len:]
            return item, j.type
            break
    return item, 'uncategorized'

times = []

for i in range(0, 60):
    '''converts listed matchtimes into separate variable in case i want to do something with that one day'''
    for j in range(0,60):
        if j < 10:
            times.append("("+str(i)+":0"+str(j)+")")
        else:
            times.append("("+str(i)+":"+str(j)+")")

### All below functions split matches into winners and losers depending on the match structure           
            
def singles_splitter(item):
    '''
    
    '''
    check = re.split('defeats', item)
    check = [item.rstrip().lstrip() for item in check]
    for item in check:
        if any(tme in item for tme in times):
            match_len = item[-7:]
            fixed = item[:-7]
            del check[-1]
            check.append(fixed)
            check.append(match_len.rstrip().lstrip())
            break
    return check

def multiway_splitter(item):
    check = re.split(' and ', item)
    check = [item.rstrip().lstrip() for item in check]
    for item in check:
        if any(tme in item for tme in times):
            match_len = item[-7:]
            fixed = item[:-7]
            del check[-1]
            check.append(fixed)
            check.append(match_len.rstrip().lstrip())
            break
    return check

def tag_splitter(item):
    check = re.split('defeat', item)
    check = [item.rstrip().lstrip() for item in check]
    for item in check:
        if any(tme in item for tme in times):
            match_len = item[-7:]
            fixed = item[:-7]
            del check[-1]
            check.append(fixed)
            check.append(match_len.rstrip().lstrip())
            break
    return check

def tag_team_parser(i):
    if re.search("\(", i):
        i = i.replace('(', ')')
        i = i.split(')')
        return(i[1])
    else:
        return(i)
    
def notes_stripper(item):
    '''removes misc bugs and issues'''
    item = item.replace(r" by referee's decision","")
    item = item.replace(r" by referee's decision","")
    item = item.replace(r" by Count Out","")
    item = item.replace(r" by Count Out","")
    item = item.replace(r" [2:1] - Kenny Omega pins PAC [2:1]","")
    return item

#### scraping cagematch for AEW events

In [348]:
webpage = 'https://www.cagematch.net/?id=8&nr=2287&page=4&s='
pages = ['0', '100', '200', '300']

all_events = []

for page in pages:
    cagematch_html = requests.get(str(webpage + page), headers={'Accept-Encoding': 'identity'})
    cagematch_html.encoding = cagematch_html.apparent_encoding
    soup = BeautifulSoup(cagematch_html.text, features='html.parser')
    links = []

    for link in soup.find_all('a'):
        links.append(link.get('href'))
        
    links2 = [item for item in links if item and '?id=1&' in item]
    links2 = [item for item in links2 if item and '&page=2' not in item]
    links2 = [item for item in links2 if item and 'view' not in item]
    for item in links2:
        all_events.append(item)

In [349]:
#test

webpage = 'https://www.cagematch.net/?id=1&nr=283430'

cagematch_html = requests.get(webpage, headers={'Accept-Encoding': 'identity'})
cagematch_html.encoding = cagematch_html.apparent_encoding
soup = BeautifulSoup(cagematch_html.text, features='html.parser')

soup.get_text()

'\n\n\n\n\n\n\n\n\n\n\nAEW Dynamite #45 « Events Database « CAGEMATCH - The Internet Wrestling Database\n\nwindow.onload = function() { loadComments("commentBox", "1", "283430", "", "en"); }\n\n\n\nNot logged in or registered. | Log In | Register | Password lost?\n\n\n\n\n\nAbout UsContactCAGEMATCH TeamYouTubeTwitterPatreon\n\nWrestlersOverviewWrestlersGimmicksTop ListsCommentsChangesEventsOverviewResultsTV + PPVUpcoming ShowsTop ListsSearch For EventsCommentsChangesMatchesOverviewMatchguideTop ListsSearch For EventsStatisticsCommentsChangesPromotionsOverviewPromotionsPromotion NamesTop ListsCommentsChangesTitlesOverviewTitle NamesTitle ChangesTop ListsCommentsChangesDatabasesTag TeamsStablesPromosFeudsRivalriesTournamentsAwardsBooksDVDsWrestling GamesMoviesThemesOtherSOTWArticlesPodcastsDocumentariesNewsboardChanges TrackerRandom ItemRatings CenterRatings System RulesHelp\n\n\n\n\n\n\nCAGEMATCH » \nEvents Database » \nAEW Dynamite #45\n\n\nAEW Dynamite #45TV-ShowResultsCardCard with g

#### Initial parse of results

In [350]:
event_titles = []
match_list = []

event_card = pd.DataFrame(columns=['Event', 'Date', 'Card'])

f = open("all_events_external.txt", "r")
parsed_events = f.read()
f.close()

parsed_events = parsed_events.split(',')[:-1]

print(parsed_events)

for page in all_events:
    if page in parsed_events:
        print('already parsed:' + page)
        continue
    else: 
        time.sleep(2)
        webpage = 'https://www.cagematch.net/' + page

        cagematch_html = requests.get(webpage, headers={'Accept-Encoding': 'identity'})
        cagematch_html.encoding = cagematch_html.apparent_encoding
        soup = BeautifulSoup(cagematch_html.text, features='html.parser')

        text = soup.get_text()

        x = re.split('::::', text)

        z = re.split('«', x[0])
        event = z[0].replace('\n', '')
        event = event.rstrip()

        if event != 'Card':
            print('now parsing:' + page)
            try:
                date = re.search('Broadcast date\:(\d\d.\d\d.\d\d\d\d)', z[2]).group(1)
            except AttributeError:
                date = re.search('Date\:(\d\d.\d\d.\d\d\d\d)', z[2]).group(1)

            event_titles.append(event)
            print(event)
            print(date)

            y = [item for item in x if 'defeat' in item] #DoubleDQ, DQ, time limit draw

            sub_list = ['(c)', ' - TITLE CHANGE !!!', '!!!', 'TITLE CHANGE']

            for sub in sub_list:
                y = [item.replace(sub, '') for item in y]

            splits = ['All workers', 'NotesThis text is hidden', 'Info:']

            for split in splits:
                y = [item.split(split, 1)[0] for item in y]

            for item in y:
                match_list.append(item)
                print(item)

            event_card = event_card.append({'Event': event, 'Date': date, 'Card': y}, ignore_index=True)
            
            f = open("all_events_external.txt", "a")
            f.write(page + ',')
            f.close()
        else:
            print('future event:' + page)

[]
future event:?id=1&nr=306030
future event:?id=1&nr=257337
future event:?id=1&nr=254319
future event:?id=1&nr=306029
future event:?id=1&nr=308232
future event:?id=1&nr=308231
future event:?id=1&nr=255225
future event:?id=1&nr=310021
future event:?id=1&nr=311408
future event:?id=1&nr=311411
now parsing:?id=1&nr=309228
AEW Dynamite #84 - Blood & Guts
05.05.2021
Tag Team MatchEddie Kingston & Jon Moxley defeat Kenny Omega & MT Nakazawa (8:03)
Singles MatchCody Rhodes (w/Arn Anderson) defeats QT Marshall (11:45)
Singles MatchDr. Britt Baker DMD (w/Rebel) defeats Julia Hart (1:30)
AEW World Tag Team Title #1 Contendership Four Way Tag Team MatchSCU (Christopher Daniels & Frankie Kazarian) defeat The Varsity Blonds (Brian Pillman Jr. & Griff Garrison) and The Jurassic Express (Jungle Boy & Luchasaurus) (w/Marko Stunt) and The Acclaimed (Anthony Bowens & Max Caster) (9:12)
Blood And Guts Ten Man Tag Team MatchThe Pinnacle (Cash Wheeler, Dax Harwood, MJF, Shawn Spears & Wardlow) (w/Tully Bla

now parsing:?id=1&nr=308860
AEW Dark #85
20.04.2021
Singles MatchJoey Janela (w/Sonny Kiss) defeats Will Allday (5:32)
Tag Team MatchSCU (Christopher Daniels & Frankie Kazarian) defeat John Skyler & Ryzin (6:54)
Tag Team MatchBig Swole & KiLynn King (w/Red Velvet) defeat Ashley MK & Steff MK (5:24)
Tag Team MatchThe Butcher And The Blade (The Blade & The Butcher) (w/Isiah Kassidy, Marq Quen, Matt Hardy & The Bunny) defeat Brick Aldridge & Cole Karter (3:40)
Singles MatchSonny Kiss defeats Jake Manning (0:52)
Six Man Tag Team MatchThe Dark Order (Evil Uno, Stu Grayson & Ten) (w/Negative One) defeat Jay Lyon, Ken Broadway & Midas Black (5:17)
Singles MatchBilly Gunn (w/Austin Gunn & Colten Gunn) defeats Andrew Palace (1:48)
Singles MatchDante Martin defeats Fuego Del Sol (4:48)
Singles MatchDiamante defeats Queen Aminata (3:18)
Tag Team MatchThe Varsity Blonds (Brian Pillman Jr. & Griff Garrison) defeat Hayden Backlund & Kit Sackett (4:42)
Singles MatchMax Caster (w/Anthony Bowens) defea

now parsing:?id=1&nr=308122
AEW Dark #83
06.04.2021
Six Man Tag Team MatchThe Dark Order (Alex Reynolds, Evil Uno & Stu Grayson) defeat Chandler Hopkins, Jake St. Patrick & Sage Scott (3:00)
Six Man Tag Team MatchTeam Taz (Brian Cage, Powerhouse Hobbs & Ricky Starks) (w/Hook) defeat Brick Aldridge, Hayden Backlund & Justin Law (4:12)
Tag Team MatchThe Jurassic Express (Jungle Boy & Luchasaurus) (w/Marko Stunt) defeat Dean Alexander & Rex Lawless (1:41)
Singles MatchRyan Nemeth (w/JD Drake) defeats Fuego Del Sol (6:06)
Singles MatchThe Butcher (w/Isiah Kassidy, Marq Quen, The Blade & The Bunny) defeats Jon Cruz (0:40)
Tag Team MatchThe Sea Stars (Ashley Vox & Delmi Exo) defeat Jazmin Allure & Vertvixen (4:31)
Tag Team MatchKiLynn King & Red Velvet (w/Big Swole) defeat Madi Wrenkowski & Vipress (5:30)
Singles MatchJD Drake (w/Ryan Nemeth) defeats Baron Black (5:16)
Tag Team MatchJoey Janela & Sonny Kiss defeat Aaron Frye & KC Navarro (2:52)
Singles MatchMatt Sydal (w/Mike Sydal) defeats 

now parsing:?id=1&nr=306008
AEW Dark #80
16.03.2021
Singles MatchLuchasaurus defeats Cezar Bononi (w/Peter Avalon) (6:29)
Singles MatchLeyla Hirsch defeats Savannah Evans (3:19)
Singles MatchJohn Silver (w/Negative One) defeats John Skyler (3:11)
Singles MatchPenelope Ford (w/Kip Sabian) defeats Tesha Price (3:24)
Tag Team MatchChaos Project (Luther & Serpentico) defeat Fuego Del Sol & Jake St. Patrick (2:48)
Singles MatchKiLynn King defeats Jazmin Allure (3:35)
Eight Man Tag Team MatchThe Dark Order (Colt Cabana, Evil Uno, Five & Stu Grayson) (w/Negative One) defeat Angel Fashion, Baron Black, Ryzin & Vary Morales (3:56)
Six Man Tag Team MatchThe Gunn Club (Austin Gunn, Billy & Colten Gunn) defeat Adam Priest, David Ali & Seth Gargis (1:01)
Tag Team MatchBear Country (Bear Boulder & Bear Bronson) defeat Brick Aldridge & Dean Alexander (0:59)
Singles MatchNick Comoroto (w/QT Marshall) defeats D3 (2:00)
Tag Team MatchThe Varsity Blonds (Brian Pillman Jr. & Griff Garrison) defeat Jorel N

now parsing:?id=1&nr=300951
AEW Dynamite #74
24.02.2021
Singles MatchJon Moxley defeats Ryan Nemeth (2:44)
Tag Team MatchTeam Taz (Brian Cage & Ricky Starks) defeat The Varsity Blonds (Brian Pillman Jr. & Griff Garrison) (9:11)
Singles MatchJake Hager defeats Brandon Cutler (3:12)
Singles MatchAdam Page defeats Isiah Kassidy (w/Angelico, Jack Evans & Matt Hardy) (13:25)
AEW Women's World Title Eliminator Tournament United States Bracket Semi Final MatchNyla Rose defeats Dr. Britt Baker DMD (w/Rebel) (12:40)
AEW TNT Title #1 Contendership Face Of The Revolution Ladder Qualifying MatchLance Archer (w/Jake Roberts) defeats Rey Fenix (17:34)
now parsing:?id=1&nr=304680
AEW Dark #77
02.03.2021
Tag Team MatchThe Nightmare Family (Aaron Solow & Lee Johnson) (w/Arn Anderson) defeat Chris Peaks & Louie Valle (5:02)
Tag Team MatchKiLynn King & Red Velvet defeat Diamante & Ivelisse (7:09)
Tag Team MatchTop Flight (Darius Martin & Daunte Martin) defeat Fuego Del Sol & Jon Cruz (5:35)
Six Man Tag T

...

now parsing:?id=1&nr=269682
AEW Dark #35
26.05.2020
Tag Team MatchThe Natural Nightmares (Dustin Rhodes & QT Marshall) (w/Brandi Rhodes) defeat The Dark Order (Alex Reynolds & John Silver) (5:01)
Singles MatchMichael Nakazawa defeats Brandon Cutler by Count Out (6:19)
Singles MatchJungle Boy (w/Luchasaurus & Marko Stunt) defeats Peter Avalon (w/Leva Bates) (8:31)
Singles MatchChristopher Daniels defeats Serpentico (4:25)
Tag Team MatchJoey Janela & Sonny Kiss defeat Brady Pierce & John Skyler (4:36)
Tag Team MatchJimmy Havoc & Kip Sabian (w/Penelope Ford) defeat Faboo Andre & Tony Donati (5:58)
Singles MatchWardlow (w/MJF) defeats Grim by referee's decision (2:56)
Singles MatchColt Cabana defeats Lee Johnson (3:49)
Singles MatchPenelope Ford (w/Kip Sabian) defeats KiLynn King (4:53)
Tag Team MatchBest Friends (Chuck Taylor & Trent) (w/Orange Cassidy) defeat Alan Angels & Shawn Dean (5:23)
now parsing:?id=1&nr=265729
AEW Dynamite #32
13.05.2020
Tag Team MatchBest Friends (Chuck Taylor &

now parsing:?id=1&nr=258710
AEW Dark #26
24.03.2020
Singles MatchJake Hager (w/Sammy Guevara) defeats Joe Alonzo (1:25)
Tag Team MatchThe Natural Nightmares (Dustin Rhodes & QT Marshall) (w/Brandi Rhodes) defeat Jon Cruz & Matt Sells (w/Skyler Moore) (4:24)
Tag Team MatchSCU (Christopher Daniels & Frankie Kazarian) defeat Robert Anthony & Shawn Spears (w/Tully Blanchard) (10:36)
Singles MatchKip Sabian (w/Penelope Ford) defeats Suge D (4:40)
Singles MatchColt Cabana defeats Brandon Cutler (4:31)
Tag Team MatchJoey Janela & Sonny Kiss defeat Corey Hollis & Mike Reed (6:09)
now parsing:?id=1&nr=251566
AEW Dynamite #23
11.03.2020
Singles MatchCody (w/Arn Anderson & Brandi Rhodes) defeats Ortiz (w/Santana) (11:35)
Tag Team MatchBea Priestley & Nyla Rose defeat Hikaru Shida & Kris Statlander (10:00)
Six Man Tag Team MatchMJF & The Butcher And The Blade (The Blade & The Butcher) (w/The Bunny & Wardlow) defeat The Jurassic Express (Jungle Boy, Luchasaurus & Marko Stunt) (11:58)
Six Man Tag Te

now parsing:?id=1&nr=254472
AEW Dark #19
11.02.2020
Non Title MatchRiho defeats Shoko Nakajima (7:34)
Singles MatchJimmy Havoc defeats Sonny Kiss (10:01)
Singles MatchHikaru Shida defeats Cassandra Golden (6:31)
Tag Team MatchThe Dark Order (Evil Uno & Stu Grayson) (w/Alex Reynolds & John Silver) defeat The Jurassic Express (Jungle Boy & Marko Stunt) (9:47)
now parsing:?id=1&nr=248029
AEW Dynamite #17
29.01.2020
Tag Team MatchThe Young Bucks (Matt Jackson & Nick Jackson) defeat The Butcher And The Blade (The Blade & The Butcher) (w/The Bunny) (8:39)
Singles MatchNyla Rose defeats Big Swole (8:44)
Singles MatchCody (w/Arn Anderson) defeats Kip Sabian (w/Penelope Ford) (12:23)
Tag Team MatchSCU (Frankie Kazarian & Scorpio Sky) (w/Christopher Daniels) defeat The Hybrid2 (Angelico & Jack Evans) (10:09)
Six Man Tag Team MatchThe Inner Circle (Chris Jericho, Ortiz & Santana) (w/Jake Hager & Sammy Guevara) defeat Darby Allin & Private Party (Isiah Kassidy & Marq Quen) (12:18)
now parsing:?id=

now parsing:?id=1&nr=238964
AEW Dynamite #9
27.11.2019
Tag Team MatchBest Friends (Chuck Taylor & Trent) (w/Orange Cassidy) defeat The Lucha Brothers (Pentagon Jr. & Rey Fenix) (7:53)
Tag Team MatchBea Priestley & Emi Sakura defeat Hikaru Shida & Kris Statlander (9:37)
Singles MatchCody defeats Matt Knicks (1:25)
Singles MatchKenny Omega defeats PAC (12:04)
Dynamite Diamond Ring MatchMJF (w/Wardlow) defeats Adam Page (7:01)
AEW World Title MatchChris Jericho  defeats Scorpio Sky (12:16)
now parsing:?id=1&nr=248342
AEW Dark #9
03.12.2019
Singles MatchJimmy Havoc defeats Brandon Cutler (7:55)
Singles MatchShawn Spears (w/Tully Blanchard) defeats Sonny Kiss (3:59)
Tag Team MatchProud-N-Powerful (Ortiz & Santana) defeat Jurassic Express (Jungle Boy & Marko Stunt) (13:26)
now parsing:?id=1&nr=240863
AEW Dynamite #8
20.11.2019
Singles MatchRey Fenix defeats Nick Jackson (11:55)
Singles MatchHikaru Shida defeats Dr. Britt Baker DMD (9:53)
Dynamite Dozen Battle RoyalAdam Page and MJF defeat Bi

now parsing:?id=1&nr=243641
AEW Dark #1
08.10.2019
Singles MatchDarby Allin defeats CIMA (6:57)
Eight Man Tag Team MatchThe Hybrid2 (Angelico & Jack Evans) & The Lucha Brothers (Pentagon Jr. & Rey Fenix) defeat Best Friends (Chuck Taylor & Trent) & Private Party (Isiah Kassidy & Marq Quen) (w/Orange Cassidy) (8:45)
Tag Team MatchAllie & Dr. Britt Baker DMD defeat Bea Priestley & Penelope Ford (10:10)
Six Man Tag Team MatchSCU (Christopher Daniels, Frankie Kazarian & Scorpio Sky) defeat The Jurassic Express (Jungle Boy, Luchasaurus & Marko Stunt) (9:16)
now parsing:?id=1&nr=231259
AEW All Out 2019
31.08.2019
Six Man Tag Team MatchSCU (Christopher Daniels, Frankie Kazarian & Scorpio Sky) defeat The Jurassic Express (Jungle Boy, Luchasaurus & Marko Stunt) (11:40)
Singles MatchPAC defeats Kenny Omega by referee's decision (23:39)
Cracker Barrel Clash Three Way MatchJimmy Havoc defeats Darby Allin and Joey Janela (14:57)
Winner Gets Bye In The AEW Tag Team Title Tournament MatchThe Dark Ord

In [351]:
event_card.to_csv('AEW_export_' + today + '.csv', index=False)

In [352]:
#event_card = pd.read_csv('AEW_export_021121.csv')
#event_card = event_card.drop(columns=['Unnamed: 0'])

In [353]:
#https://stackoverflow.com/questions/27263805/pandas-column-of-lists-create-a-row-for-each-list-element

lst_col = 'Card'

r = pd.DataFrame({
      col:np.repeat(event_card[col].values, event_card[lst_col].str.len())
      for col in event_card.columns.drop(lst_col)}
    ).assign(**{lst_col:np.concatenate(event_card[lst_col].values)})[event_card.columns]


r

,Event,Date,Card
0,AEW Dynamite #84 - Blood & Guts,05.05.2021,Tag Team MatchEddie Kingston & Jon Moxley defe...
1,AEW Dynamite #84 - Blood & Guts,05.05.2021,Singles MatchCody Rhodes (w/Arn Anderson) defe...
2,AEW Dynamite #84 - Blood & Guts,05.05.2021,Singles MatchDr. Britt Baker DMD (w/Rebel) def...
3,AEW Dynamite #84 - Blood & Guts,05.05.2021,AEW World Tag Team Title #1 Contendership Four...
4,AEW Dynamite #84 - Blood & Guts,05.05.2021,Blood And Guts Ten Man Tag Team MatchThe Pinna...
...,...,...,...
1432,AEW Double Or Nothing 2019,25.05.2019,Singles MatchCody (w/Brandi Rhodes) defeats Du...
1433,AEW Double Or Nothing 2019,25.05.2019,AAA World Tag Team Title MatchThe Young Bucks ...
1434,AEW Double Or Nothing 2019,25.05.2019,Singles MatchChris Jericho defeats Kenny Omega...
1435,AEW Double Or Nothing - The Buy In,25.05.2019,21 Man Casino Battle Royale MatchAdam Page def...


In [354]:
y = [match_parser(item) for item in r.Card]

temp1 = pd.DataFrame(columns=['results', 'matchtype'], data = y)

r['result'] = temp1.results
r['matchtype'] = temp1.matchtype

r

,Event,Date,Card,result,matchtype
0,AEW Dynamite #84 - Blood & Guts,05.05.2021,Tag Team MatchEddie Kingston & Jon Moxley defe...,Eddie Kingston & Jon Moxley defeat Kenny Omega...,tag
1,AEW Dynamite #84 - Blood & Guts,05.05.2021,Singles MatchCody Rhodes (w/Arn Anderson) defe...,Cody Rhodes (w/Arn Anderson) defeats QT Marsha...,singles
2,AEW Dynamite #84 - Blood & Guts,05.05.2021,Singles MatchDr. Britt Baker DMD (w/Rebel) def...,Dr. Britt Baker DMD (w/Rebel) defeats Julia Ha...,singles
3,AEW Dynamite #84 - Blood & Guts,05.05.2021,AEW World Tag Team Title #1 Contendership Four...,SCU (Christopher Daniels & Frankie Kazarian) d...,fourwaytag
4,AEW Dynamite #84 - Blood & Guts,05.05.2021,Blood And Guts Ten Man Tag Team MatchThe Pinna...,"The Pinnacle (Cash Wheeler, Dax Harwood, MJF, ...",tag10
...,...,...,...,...,...
1432,AEW Double Or Nothing 2019,25.05.2019,Singles MatchCody (w/Brandi Rhodes) defeats Du...,Cody (w/Brandi Rhodes) defeats Dustin Rhodes (...,singles
1433,AEW Double Or Nothing 2019,25.05.2019,AAA World Tag Team Title MatchThe Young Bucks ...,The Young Bucks (Matt Jackson & Nick Jackson) ...,tag
1434,AEW Double Or Nothing 2019,25.05.2019,Singles MatchChris Jericho defeats Kenny Omega...,Chris Jericho defeats Kenny Omega (26:52),singles
1435,AEW Double Or Nothing - The Buy In,25.05.2019,21 Man Casino Battle Royale MatchAdam Page def...,Adam Page defeats Ace Romero and Billy Gunn an...,br


In [355]:
r_master = pd.read_csv('AEW_export_r_master.csv')
r_master

In [356]:
r = pd.concat([r, r_master])
r.to_csv('AEW_export_r_master.csv', index=False)

#### categorizing matches

In [357]:
r = pd.read_csv('AEW_export_r_master.csv')

In [358]:
r[r.matchtype=='uncategorized']

,Event,Date,Card,result,matchtype
69,AEW The House Always Wins,09.04.2021,AEW TNT Title #1 Contendership 15 Man Battle R...,AEW TNT Title #1 Contendership 15 Man Battle R...,uncategorized


In [359]:
r.matchtype.value_counts()

singles          833
tag              435
tag6              87
tag8              24
br                10
fourway            9
threewaytag        8
threeway           7
tag10              6
fourwaytag         5
unsanctioned       4
handicap2on1       3
tag12              2
uncategorized      1
handicap3on1       1
sixway             1
nineway            1
Name: matchtype, dtype: int64

In [360]:
r = r[['Event', 'Date', 'result', 'matchtype']]

#### parsing singles matches

In [361]:
singles_matches = r[r['matchtype'] == 'singles']

singles_matches['win1'] = [singles_splitter(i)[0] for i in singles_matches['result']]
singles_matches['lose1'] = [singles_splitter(i)[1] for i in singles_matches['result']]
singles_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in singles_matches['result']]

singles_matches.win1 = [i.rstrip().lstrip() for i in singles_matches['win1'].str.replace(r"\(w\/.*?\)","")]
singles_matches.lose1 = [i.rstrip().lstrip() for i in singles_matches['lose1'].str.replace(r"\(w\/.*?\)","")]
singles_matches.lose1 = [notes_stripper(i) for i in singles_matches.lose1]

In [362]:
singles_matches[:20]

,Event,Date,result,matchtype,win1,lose1,time
1,AEW Dynamite #84 - Blood & Guts,05.05.2021,Cody Rhodes (w/Arn Anderson) defeats QT Marsha...,singles,Cody Rhodes,QT Marshall,(11:45)
2,AEW Dynamite #84 - Blood & Guts,05.05.2021,Dr. Britt Baker DMD (w/Rebel) defeats Julia Ha...,singles,Dr. Britt Baker DMD,Julia Hart,(1:30)
5,AEW Dark: Elevation #8,03.05.2021,Miro defeats Will Allday (4:32),singles,Miro,Will Allday,(4:32)
6,AEW Dark: Elevation #8,03.05.2021,Abadon defeats Ryo Mizunami (6:51),singles,Abadon,Ryo Mizunami,(6:51)
7,AEW Dark: Elevation #8,03.05.2021,Jon Moxley defeats Andrew Palace by referee's ...,singles,Jon Moxley,Andrew Palace,(3:08)
8,AEW Dark: Elevation #8,03.05.2021,Ten defeats D3 by referee's decision (1:57),singles,Ten,D3,(1:57)
9,AEW Dark: Elevation #8,03.05.2021,Orange Cassidy (w/Kris Statlander) defeats VSK...,singles,Orange Cassidy,VSK,(1:51)
11,AEW Dark: Elevation #8,03.05.2021,Nyla Rose (w/Vickie Guerrero) defeats Madi Wre...,singles,Nyla Rose,Madi Wrenkowski,(1:55)
12,AEW Dark: Elevation #8,03.05.2021,"Nick Comoroto (w/Aaron Solow, Anthony Ogogo & ...",singles,Nick Comoroto,Baron Black,(2:13)
13,AEW Dark: Elevation #8,03.05.2021,Red Velvet defeats Reka Tehaka (3:35),singles,Red Velvet,Reka Tehaka,(3:35)


#### parsing tag matches

In [363]:
tag_matches = r[r['matchtype'] == 'tag']

tag_matches['win1'] = [tag_splitter(i)[0] for i in tag_matches['result']]
tag_matches['lose1'] = [tag_splitter(i)[1] for i in tag_matches['result']]
tag_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag_matches['result']]

tag_matches.win1 = [i.rstrip().lstrip() for i in tag_matches['win1'].str.replace(r"\(w\/.*?\)","")]
tag_matches.lose1 = [i.rstrip().lstrip() for i in tag_matches['lose1'].str.replace(r"\(w\/.*?\)","")]

tag_matches['win1'] = [tag_team_parser(i) for i in tag_matches.win1]
tag_matches['lose1'] = [tag_team_parser(i) for i in tag_matches.lose1]

tag_matches[['win1','win2']] = tag_matches.win1.str.split(" & ",expand=True)
tag_matches[['lose1','lose2']] = tag_matches.lose1.str.split(" & ",expand=True) 

tag_matches = tag_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'time']]

In [364]:
tag_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,time
0,AEW Dynamite #84 - Blood & Guts,05.05.2021,Eddie Kingston & Jon Moxley defeat Kenny Omega...,tag,Eddie Kingston,Jon Moxley,Kenny Omega,MT Nakazawa,(8:03)
10,AEW Dark: Elevation #8,03.05.2021,FTR (Cash Wheeler & Dax Harwood) (w/Tully Blan...,tag,Cash Wheeler,Dax Harwood,Terrell Hughes,Terrence Hughes,(4:58)
14,AEW Dark: Elevation #8,03.05.2021,Private Party (Isiah Kassidy & Marq Quen) (w/M...,tag,Isiah Kassidy,Marq Quen,Duke Davis,Ganon Jones,(3:57)
17,AEW Dark: Elevation #8,03.05.2021,The Gunn Club (Austin Gunn & Colten Gunn) defe...,tag,Austin Gunn,Colten Gunn,Luther,Serpentico,(8:56)
18,AEW Dark: Elevation #8,03.05.2021,The Hardy Family Office (Matt Hardy & The Blad...,tag,Matt Hardy,The Blade,Colt Cabana,Five,(9:30)
...,...,...,...,...,...,...,...,...,...
1415,AEW Fight For The Fallen,13.07.2019,The Lucha Brothers (Pentagon Jr. & Rey Fenix) ...,tag,Pentagon Jr.,Rey Fenix,Frankie Kazarian,Scorpio Sky,(15:05)
1417,AEW Fight For The Fallen,13.07.2019,The Young Bucks (Matt Jackson & Nick Jackson) ...,tag,Matt Jackson,Nick Jackson,Cody,Dustin Rhodes,(31:26)
1419,AEW Fight For The Fallen - The Buy In,13.07.2019,Bea Priestley & Shoko Nakajima defeat Dr. Brit...,tag,Bea Priestley,Shoko Nakajima,Dr. Britt Baker DMD,Riho,(15:38)
1430,AEW Double Or Nothing 2019,25.05.2019,Best Friends (Chuck Taylor & Trent) defeat Ang...,tag,Chuck Taylor,Trent,Angelico,Jack Evans,(12:42)


#### parsing tag6s

In [365]:
def tag6_parser(item):
    one_and_two = '.*? & .*? \(.*\)'
    two_and_one = '.*? \(.*\) & .*'
    trio = '.*? \(.*\)'
    three_singles = '.*? \& .*'
    
    if re.match(one_and_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[2], x[3]]
        return(z)
    elif re.match(two_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[4]]
        return(z)
    elif re.match(trio, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3]]
        return(z)
    elif re.match(three_singles, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2]]
        return(z)

In [366]:
tag6_matches = r[r['matchtype'] == 'tag6']

tag6_matches['win'] = [tag_splitter(i)[0] for i in tag6_matches['result']]
tag6_matches['lose'] = [tag_splitter(i)[1] for i in tag6_matches['result']]
tag6_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag6_matches['result']]

tag6_matches.win = [i.rstrip().lstrip() for i in tag6_matches['win'].str.replace(r"\(w.*\)","")]
tag6_matches.lose = [i.rstrip().lstrip() for i in tag6_matches['lose'].str.replace(r"\(w.*\)","")]

tag6_matches.win = [tag6_parser(i) for i in tag6_matches.win]
tag6_matches.lose = [tag6_parser(i) for i in tag6_matches.lose]

tag6_matches['win1'] = [i[0] for i in tag6_matches.win]
tag6_matches['win2'] = [i[1] for i in tag6_matches.win]
tag6_matches['win3'] = [i[2] for i in tag6_matches.win]

tag6_matches['lose1'] = [i[0] for i in tag6_matches.lose]
tag6_matches['lose2'] = [i[1] for i in tag6_matches.lose]
tag6_matches['lose3'] = [i[2] for i in tag6_matches.lose]

tag6_matches = tag6_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'lose1', 'lose2', 'lose3', 'time']]


tag6_matches

,Event,Date,result,matchtype,win1,win2,win3,lose1,lose2,lose3,time
30,AEW Dark #87,04.05.2021,"The Dark Order (Alex Reynolds, Evil Uno & Stu ...",tag6,Alex Reynolds,Evil Uno,Stu Grayson,Cezar Bononi,JD Drake,Ryan Nemeth,(7:07)
37,AEW Dynamite #83,28.04.2021,"The Factory (Aaron Solow, Nick Comoroto & QT M...",tag6,Aaron Solow,Nick Comoroto,QT Marshall,Billy Gunn,Dustin Rhodes,Lee Johnson,(6:44)
62,AEW Dark #86,27.04.2021,"The Dark Order (Evil Uno, Stu Grayson & Ten) (...",tag6,Evil Uno,Stu Grayson,Ten,Isiah Kassidy,Marq Quen,The Blade,(10:07)
73,AEW The House Always Wins,09.04.2021,Best Friends (Chuck Taylor & Trent) & Orange C...,tag6,Chuck Taylor,Trent,Orange Cassidy,Angelico,Jack Evans,Max Caster,(11:10)
83,AEW Dark #85,20.04.2021,"The Dark Order (Evil Uno, Stu Grayson & Ten) (...",tag6,Evil Uno,Stu Grayson,Ten,Jay Lyon,Ken Broadway,Midas Black,(5:17)
...,...,...,...,...,...,...,...,...,...,...,...
1401,AEW All Out 2019,31.08.2019,"SCU (Christopher Daniels, Frankie Kazarian & S...",tag6,Christopher Daniels,Frankie Kazarian,Scorpio Sky,Jungle Boy,Luchasaurus,Marko Stunt,(11:40)
1411,AEW Fight For The Fallen,13.07.2019,"Maxwell Jacob Friedman, Sammy Guevara & Shawn ...",tag6,Maxwell Jacob Friedman,Sammy Guevara,Shawn Spears,Darby Allin,Jimmy Havoc,Joey Janela,(13:13)
1423,AEW Fyter Fest,29.06.2019,"The Elite (Kenny Omega, Matt Jackson & Nick Ja...",tag6,Kenny Omega,Matt Jackson,Nick Jackson,Laredo Kid,Pentagon Jr.,Rey Fenix,(20:29)
1428,AEW Double Or Nothing 2019,25.05.2019,"SCU (Christopher Daniels, Frankie Kazarian & S...",tag6,Christopher Daniels,Frankie Kazarian,Scorpio Sky,CIMA,El Lindaman,T-Hawk,(13:38)


#### parsing tag8s

In [367]:
def tag8_parser(item):
    two_and_two = '.*? \(.*\) & .*? \(.*\)'
    two_and_two_error = '.*? \(.*\)  & .*? \(.*\)'
    three_and_one = '.*? \(.*\) & .*'
    one_one_two = '.*,.* & .*? \(.*\)'
    two_one_one = '.*\(.*\),.*&.*'
    one_and_three = '.* & .*? \(.*\)'
    quad = '.*? \(.*\)'
    
    if re.match(two_and_two, item) or re.match(two_and_two_error, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6]]
        return(z)
    elif re.match(two_one_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[4], x[5]]
        return(z)
    elif re.match(three_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[5]]
        return(z)
    elif re.match(one_one_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[3], x[4]]
        return(z)
    elif re.match(one_and_three, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[2], x[3], x[4]]
        return(z)
    elif re.match(quad, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4]]
        return(z)
    else:
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2], x[3]]
        return(z)

In [368]:
tag8_matches = r[r['matchtype'] == 'tag8']

tag8_matches['win'] = [tag_splitter(i)[0] for i in tag8_matches['result']]
tag8_matches['lose'] = [tag_splitter(i)[1] for i in tag8_matches['result']]
tag8_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag8_matches['result']]

tag8_matches.win = [i.rstrip().lstrip() for i in tag8_matches['win'].str.replace(r"\(w\/.*?\)","")]
tag8_matches.lose = [i.rstrip().lstrip() for i in tag8_matches['lose'].str.replace(r"\(w\/.*?\)","")]

tag8_matches.win = [tag8_parser(i) for i in tag8_matches.win]
tag8_matches.lose = [tag8_parser(i) for i in tag8_matches.lose]

tag8_matches['win1'] = [i[0] for i in tag8_matches.win]
tag8_matches['win2'] = [i[1] for i in tag8_matches.win]
tag8_matches['win3'] = [i[2] for i in tag8_matches.win]
tag8_matches['win4'] = [i[3] for i in tag8_matches.win]

tag8_matches['lose1'] = [i[0] for i in tag8_matches.lose]
tag8_matches['lose2'] = [i[1] for i in tag8_matches.lose]
tag8_matches['lose3'] = [i[2] for i in tag8_matches.lose]
tag8_matches['lose4'] = [i[3] for i in tag8_matches.lose]


tag8_matches = tag8_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'win4', 'lose1', 'lose2', 'lose3', 'lose4', 'time']]


tag8_matches


,Event,Date,result,matchtype,win1,win2,win3,win4,lose1,lose2,lose3,lose4,time
70,AEW The House Always Wins,09.04.2021,"The Pinnacle (Cash Wheeler, Dax Harwood, MJF &...",tag8,Cash Wheeler,Dax Harwood,MJF,Shawn Spears,Christopher Daniels,Dante Martin,Jungle Boy,Luchasaurus,(14:30)
75,AEW The House Always Wins,09.04.2021,"Hikaru Shida, Red Velvet, Ryo Mizunami & Tay C...",tag8,Hikaru Shida,Red Velvet,Ryo Mizunami,Tay Conti,Dr. Britt Baker DMD,Nyla Rose,Rebel,The Bunny,(14:00)
163,AEW Dark: Elevation #4,05.04.2021,The Butcher And The Blade (The Blade & The But...,tag8,The Blade,The Butcher,Isiah Kassidy,Marq Quen,Adam Priest,D3,Fuego Del Sol,Ryzin,(4:30)
258,AEW Dark: Elevation #1,15.03.2021,The Butcher And The Blade (The Blade & The But...,tag8,The Blade,The Butcher,Isiah Kassidy,Marq Quen,Brick Aldridge,Carlie Bravo,David Ali,Dean Alexander,(4:32)
267,AEW Dark #80,16.03.2021,"The Dark Order (Colt Cabana, Evil Uno, Five & ...",tag8,Colt Cabana,Evil Uno,Five,Stu Grayson,Angel Fashion,Baron Black,Ryzin,Vary Morales,(3:56)
307,AEW Dark #79,09.03.2021,"The Dark Order (Alex Reynolds, Colt Cabana, Ev...",tag8,Alex Reynolds,Colt Cabana,Evil Uno,Stu Grayson,Aaron Frye,D3,Jon Cruz,Vary Morales,(5:41)
363,AEW Dark #76,23.02.2021,"Bear Country (Bear Boulder & Bear Bronson), Jo...",tag8,Bear Boulder,Bear Bronson,Joey Janela,Sonny Kiss,Aaron Frye,Daniel Joseph,Levy Shapiro,M'Badu,(5:24)
411,AEW Dynamite #70,27.01.2021,The Good Brothers (Karl Anderson & Luke Gallow...,tag8,Karl Anderson,Luke Gallows,Matt Jackson,Nick Jackson,Alex Reynolds,Evil Uno,John Silver,Stu Grayson,(11:46)
419,AEW Dark #73,02.02.2021,"Bear Country (Bear Boulder & Bear Bronson), Jo...",tag8,Bear Boulder,Bear Bronson,Joey Janela,Sonny Kiss,Aaron Solow,Baron Black,Mike Verna,Shawn Dean,(5:15)
425,AEW Dynamite #69,20.01.2021,"Adam Page & The Dark Order (Alex Reynolds, Col...",tag8,Adam Page,Alex Reynolds,Colt Cabana,John Silver,Luther,Serpentico,Angelico,Jack Evans,(6:20)


#### parsing tag10s

In [369]:
def tag10_parser(item):
    four_and_one = '.*? \(.*\) & .*'
    three_one_one = '.*? \(.*\), .*? & .*'
    two_one_one_one = '.*? \(.*\), .*?, .*? & .*'
    two_two_one = '.*? \(.*\), .*? \(.*\) & .*'
    two_three = '.*? \(.*?\) & .*?\(.*?\)'
    fivesome = '.*? \(.*\)'
    one_two_two = '.*?, .*? \(.*\) &.*? \(.*\)'
    
    if re.match(two_two_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6], x[8]]
        return(z)
    elif re.match(two_one_one_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[4], x[5], x[6]]
        return(z)
    elif re.match(one_two_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[2], x[3], x[6], x[7]]
        return(z)
    elif re.match(three_one_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[5], x[6]]
        return(z)
    elif re.match(two_three, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6], x[7]]
        return(z)
    elif re.match(four_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[6]]
        return(z)
    elif re.match(fivesome, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[5]]
        return(z)
    else:
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2], x[3], x[4]]
        return(z)

In [370]:
tag10_matches = r[r['matchtype'] == 'tag10']

tag10_matches['win'] = [tag_splitter(i)[0] for i in tag10_matches['result']]
tag10_matches['lose'] = [tag_splitter(i)[1] for i in tag10_matches['result']]
tag10_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag10_matches['result']]

tag10_matches.win = [i.rstrip().lstrip() for i in tag10_matches['win'].str.replace(r"\(w\/.*?\)","")]
tag10_matches.lose = [i.rstrip().lstrip() for i in tag10_matches['lose'].str.replace(r"\(w\/.*?\)","")]

tag10_matches.win = [tag10_parser(i) for i in tag10_matches.win]
tag10_matches.lose = [tag10_parser(i) for i in tag10_matches.lose]

tag10_matches['win1'] = [i[0] for i in tag10_matches.win]
tag10_matches['win2'] = [i[1] for i in tag10_matches.win]
tag10_matches['win3'] = [i[2] for i in tag10_matches.win]
tag10_matches['win4'] = [i[3] for i in tag10_matches.win]
tag10_matches['win5'] = [i[4] for i in tag10_matches.win]

tag10_matches['lose1'] = [i[0] for i in tag10_matches.lose]
tag10_matches['lose2'] = [i[1] for i in tag10_matches.lose]
tag10_matches['lose3'] = [i[2] for i in tag10_matches.lose]
tag10_matches['lose4'] = [i[3] for i in tag10_matches.lose]
tag10_matches['lose5'] = [i[4] for i in tag10_matches.lose]

tag10_matches = tag10_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'win4', 'win5', 'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'time']]

tag10_matches

,Event,Date,result,matchtype,win1,win2,win3,win4,win5,lose1,lose2,lose3,lose4,lose5,time
4,AEW Dynamite #84 - Blood & Guts,05.05.2021,"The Pinnacle (Cash Wheeler, Dax Harwood, MJF, ...",tag10,Cash Wheeler,Dax Harwood,MJF,Shawn Spears,Wardlow,Chris Jericho,Jake Hager,Ortiz,Sammy Guevara,Santana,(17:00)
76,AEW The House Always Wins,09.04.2021,"Death Triangle (PAC, Penta El Zero Miedo & Rey...",tag10,PAC,Penta El Zero Miedo,Rey Fenix,Matt Sydal,Mike Sydal,Matt Jackson,Nick Jackson,Kenny Omega,Konosuke Takeshita,Michael Nakazawa,(12:25)
214,AEW Dynamite #77 - St. Patrick's Day Slam,17.03.2021,"Matt Hardy, Private Party (Isiah Kassidy & Mar...",tag10,Matt Hardy,Isiah Kassidy,Marq Quen,The Blade,The Butcher,Bear Boulder,Bear Bronson,Jungle Boy,Luchasaurus,Marko Stunt,(8:47)
475,AEW Dark #70,12.01.2021,"The Dark Order (Colt Cabana, Evil Uno, Five, S...",tag10,Colt Cabana,Evil Uno,Five,Stu Grayson,Ten,Adam Priest,Danny Limelight,Shawn Dean,Vary Morales,Zack Clayton,(8:19)
920,AEW Dynamite #43,29.07.2020,"Best Friends (Chuck Taylor & Trent), The Juras...",tag10,Chuck Taylor,Trent,Jungle Boy,Luchasaurus,Orange Cassidy,Chris Jericho,Jake Hager,Ortiz,Sammy Guevara,Santana,(12:06)
1069,AEW Double Or Nothing 2020,23.05.2020,"The Elite (Adam Page, Kenny Omega, Matt Jackso...",tag10,Adam Page,Kenny Omega,Matt Jackson,Nick Jackson,Matt Hardy,Chris Jericho,Jake Hager,Ortiz,Sammy Guevara,Santana,(33:59)


#### parsing tag12s

In [371]:
def tag12_parser(item):
    four_and_two = '.*? \(.*\) & .* \(.*\)'
    five_and_one = '.*? \(.*\) & .*'
    two_two_two =  '.*? \(.*\), .*? \(.*\) & .*? \(.*\)'
    sixsome = '.*? \(.*\)'
    
    if re.match(two_two_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[5], x[6], x[9], x[10]]
        return(z)
    elif re.match(four_and_two, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[7], x[8]]
        return(z)
    elif re.match(five_and_one, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[5], x[7]]
        return(z)
    elif re.match(sixsome, item):
        x = re.split(r'[,&()]\s*', item)
        z = [x[1], x[2], x[3], x[4], x[5], x[6]]
        return(z)
    else:
        x = re.split(r'[,&()]\s*', item)
        z = [x[0], x[1], x[2], x[3], x[4]]
        return(z)

In [372]:
tag12_matches = r[r['matchtype'] == 'tag12']

tag12_matches['win'] = [tag_splitter(i)[0] for i in tag12_matches['result']]
tag12_matches['lose'] = [tag_splitter(i)[1] for i in tag12_matches['result']]
tag12_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in tag12_matches['result']]

tag12_matches.win = [i.rstrip().lstrip() for i in tag12_matches['win'].str.replace(r"\(w\/.*?\)","")]
tag12_matches.lose = [i.rstrip().lstrip() for i in tag12_matches['lose'].str.replace(r"\(w\/.*?\)","")]

tag12_matches.win = [tag12_parser(i) for i in tag12_matches.win]
tag12_matches.lose = [tag12_parser(i) for i in tag12_matches.lose]

tag12_matches['win1'] = [i[0] for i in tag12_matches.win]
tag12_matches['win2'] = [i[1] for i in tag12_matches.win]
tag12_matches['win3'] = [i[2] for i in tag12_matches.win]
tag12_matches['win4'] = [i[3] for i in tag12_matches.win]
tag12_matches['win5'] = [i[4] for i in tag12_matches.win]
tag12_matches['win6'] = [i[5] for i in tag12_matches.win]

tag12_matches['lose1'] = [i[0] for i in tag12_matches.lose]
tag12_matches['lose2'] = [i[1] for i in tag12_matches.lose]
tag12_matches['lose3'] = [i[2] for i in tag12_matches.lose]
tag12_matches['lose4'] = [i[3] for i in tag12_matches.lose]
tag12_matches['lose5'] = [i[4] for i in tag12_matches.lose]
tag12_matches['lose6'] = [i[5] for i in tag12_matches.lose]

tag12_matches = tag12_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'win3', 'win4', 'win5', 'win6', 'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'time']]


tag12_matches

,Event,Date,result,matchtype,win1,win2,win3,win4,win5,win6,lose1,lose2,lose3,lose4,lose5,lose6,time
540,AEW Dynamite #64,16.12.2020,"The Inner Circle (Chris Jericho, Jake Hager, M...",tag12,Chris Jericho,Jake Hager,MJF,Ortiz,Sammy Guevara,Santana,Chuck Taylor,Trent,Brian Pillman Jr.,Griff Garrison,Darius Martin,Daunte Martin,(14:19)
906,AEW Dynamite #44,05.08.2020,"The Dark Order (Evil Uno, Five, Mr. Brodie Lee...",tag12,Evil Uno,Five,Mr. Brodie Lee,Nine,Stu Grayson,Colt Cabana,Adam Page,Kenny Omega,Matt Jackson,Nick Jackson,Cash Wheeler,Dax Harwood,(17:19)


#### parsing fourways

In [373]:
fourway_matches = r[r['matchtype'] == 'fourway']


fourway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in fourway_matches['result']]
fourway_matches.result = [i.rstrip().lstrip() for i in fourway_matches['result'].str.replace(r"\(w\/.*?\)","")]

fourway_matches['win1'] = [singles_splitter(i)[0] for i in fourway_matches['result']]
fourway_matches['lose'] = [singles_splitter(i)[1] for i in fourway_matches['result']]
fourway_matches[['lose1', 'lose2', 'lose3']] = [(multiway_splitter(i)[0], multiway_splitter(i)[1], multiway_splitter(i)[2]) for i in fourway_matches.lose]


fourway_matches = fourway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', 'lose3', 'time']]


fourway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3,time
502,AEW Dark #69,05.01.2021,Frankie Kazarian defeats Angelico and Darius...,fourway,Frankie Kazarian,Angelico,Darius Martin,Griff Garrison,(7:43)
745,AEW Dark #56,13.10.2020,Jungle Boy defeats Evil Uno and Frankie Kazari...,fourway,Jungle Boy,Evil Uno,Frankie Kazarian,The Blade,(8:29)
1087,AEW Dynamite #32,13.05.2020,Hikaru Shida defeats Dr. Britt Baker DMD and K...,fourway,Hikaru Shida,Dr. Britt Baker DMD,Kris Statlander,Penelope Ford,(10:12)
1170,AEW Dynamite #24,18.03.2020,Hikaru Shida defeats Kris Statlander and Penel...,fourway,Hikaru Shida,Kris Statlander,Penelope Ford,Riho,(5:49)
1209,AEW Dynamite #21,26.02.2020,Hikaru Shida defeats Big Swole and Shanna and ...,fourway,Hikaru Shida,Big Swole,Shanna,Yuka Sakazaki,(9:13)
1275,AEW Dynamite #13 - Homecoming Edition,01.01.2020,Riho defeats Dr. Britt Baker DMD and Hikaru S...,fourway,Riho,Dr. Britt Baker DMD,Hikaru Shida,Nyla Rose,(9:49)
1373,AEW Dark #4,29.10.2019,Emi Sakura defeats Allie and Penelope Ford and...,fourway,Emi Sakura,Allie,Penelope Ford,Sadie Gibbs,(11:54)
1422,AEW Fyter Fest,29.06.2019,Adam Page defeats Jimmy Havoc and Jungle Boy ...,fourway,Adam Page,Jimmy Havoc,Jungle Boy,Maxwell Jacob Friedman,(10:51)
1429,AEW Double Or Nothing 2019,25.05.2019,Dr. Britt Baker DMD defeats Awesome Kong and ...,fourway,Dr. Britt Baker DMD,Awesome Kong,Kylie Rae,Nyla Rose,(11:08)


#### parsing threeways

In [374]:
threeway_matches = r[r['matchtype'] == 'threeway']


threeway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in threeway_matches['result']]
threeway_matches.result = [i.rstrip().lstrip() for i in threeway_matches['result'].str.replace(r"\(w\/.*?\)","")]

threeway_matches['win1'] = [singles_splitter(i)[0] for i in threeway_matches['result']]
threeway_matches['lose'] = [singles_splitter(i)[1] for i in threeway_matches['result']]
threeway_matches[['lose1', 'lose2']] = [(multiway_splitter(i)[0], multiway_splitter(i)[1]) for i in threeway_matches.lose]


threeway_matches = threeway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', 'time']]


threeway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,time
1159,AEW @The BTE Compound,13.04.2020,Scorpio Sky defeats Brandon Cutler and Peter A...,threeway,Scorpio Sky,Brandon Cutler,Peter Avalon,(Unk.)
1312,AEW Dark #10,10.12.2019,Scorpio Sky defeats Jimmy Havoc and Peter Aval...,threeway,Scorpio Sky,Jimmy Havoc,Peter Avalon,(6:28)
1336,AEW Dynamite #7,13.11.2019,Darby Allin defeats Peter Avalon and Shawn Sp...,threeway,Darby Allin,Peter Avalon,Shawn Spears,(3:44)
1374,AEW Dark #4,29.10.2019,Darby Allin defeats Jack Evans and Jimmy Havoc...,threeway,Darby Allin,Jack Evans,Jimmy Havoc,(13:12)
1389,AEW Dark #2,15.10.2019,Kip Sabian defeats Peter Avalon and Sonny Kis...,threeway,Kip Sabian,Peter Avalon,Sonny Kiss,(6:06)
1403,AEW All Out 2019,31.08.2019,Jimmy Havoc defeats Darby Allin and Joey Janel...,threeway,Jimmy Havoc,Darby Allin,Joey Janela,(14:57)
1421,AEW Fyter Fest,29.06.2019,Riho defeats Nyla Rose and Yuka Sakazaki (13:02),threeway,Riho,Nyla Rose,Yuka Sakazaki,(13:02)


#### parsing fourwaytags

In [375]:
fourwaytag_matches = r[r['matchtype'] == 'fourwaytag']
fourwaytag_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in fourwaytag_matches['result']]
fourwaytag_matches.result = [i.rstrip().lstrip() for i in fourwaytag_matches['result'].str.replace(r"\(w\/.*?\)","")]

fourwaytag_matches['win_team'] = [tag_splitter(i)[0] for i in fourwaytag_matches['result']]
fourwaytag_matches['lose_teams'] = [tag_splitter(i)[1] for i in fourwaytag_matches['result']]
fourwaytag_matches[['loseteam1', 'loseteam2', 'loseteam3']] = [(multiway_splitter(i)[0], multiway_splitter(i)[1], multiway_splitter(i)[2]) for i in fourwaytag_matches.lose_teams]

fourwaytag_matches['win_team'] = [tag_team_parser(i) for i in fourwaytag_matches.win_team]
fourwaytag_matches['loseteam1'] = [tag_team_parser(i) for i in fourwaytag_matches.loseteam1]
fourwaytag_matches['loseteam2'] = [tag_team_parser(i) for i in fourwaytag_matches.loseteam2]
fourwaytag_matches['loseteam3'] = [tag_team_parser(i) for i in fourwaytag_matches.loseteam3]

fourwaytag_matches[['win1', 'win2']] = fourwaytag_matches.win_team.str.split(" & ",expand=True)
fourwaytag_matches[['lose1', 'lose2']] = fourwaytag_matches.loseteam1.str.split(" & ",expand=True) 
fourwaytag_matches[['lose3', 'lose4']] = fourwaytag_matches.loseteam2.str.split(" & ",expand=True) 
fourwaytag_matches[['lose5', 'lose6']] = fourwaytag_matches.loseteam3.str.split(" & ",expand=True) 

fourwaytag_matches = fourwaytag_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'time']]


fourwaytag_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,lose3,lose4,lose5,lose6,time
3,AEW Dynamite #84 - Blood & Guts,05.05.2021,SCU (Christopher Daniels & Frankie Kazarian) d...,fourwaytag,Christopher Daniels,Frankie Kazarian,Brian Pillman Jr.,Griff Garrison,Jungle Boy,Luchasaurus,Anthony Bowens,Max Caster,(9:12)
704,AEW Dynamite #56,21.10.2020,The Young Bucks (Matt Jackson & Nick Jackson) ...,fourwaytag,Matt Jackson,Nick Jackson,Isiah Kassidy,Marq Quen,The Blade,The Butcher,Alex Reynolds,John Silver,(13:29)
855,AEW Dynamite #47 - Thursday Night Dynamite,27.08.2020,FTR (Cash Wheeler & Dax Harwood) defeat Best ...,fourwaytag,Cash Wheeler,Dax Harwood,Chuck Taylor,Trent,Dustin Rhodes,QT Marshall,Matt Jackson,Nick Jackson,(5:25)
1255,AEW Dynamite #15 - Bash At The Beach,15.01.2020,Adam Page & Kenny Omega defeat Best Friends (C...,fourwaytag,Adam Page,Kenny Omega,Chuck Taylor,Trent,Ortiz,Santana,Matt Jackson,Nick Jackson,(16:35)
1294,AEW Dark #12,24.12.2019,Proud-N-Powerful (Ortiz & Santana) defeat Best...,fourwaytag,Ortiz,Santana,Chuck Taylor,Trent,Isiah Kassidy,Marq Quen,Angelico,Jack Evans,(16:08)


#### parsing threewaytags

In [376]:
threewaytag_matches = r[r['matchtype'] == 'threewaytag']
threewaytag_matches['time'] = [tag_splitter(i)[2] if len(tag_splitter(i)) == 3 else '(Unk.)' for i in threewaytag_matches['result']]
threewaytag_matches.result = [i.rstrip().lstrip() for i in threewaytag_matches['result'].str.replace(r"\(w\/.*?\)","")]

threewaytag_matches['win_team'] = [tag_splitter(i)[0] for i in threewaytag_matches['result']]
threewaytag_matches['lose_teams'] = [tag_splitter(i)[1] for i in threewaytag_matches['result']]
threewaytag_matches[['loseteam1', 'loseteam2']] = [(multiway_splitter(i)[0], multiway_splitter(i)[1]) for i in threewaytag_matches.lose_teams]

threewaytag_matches['win_team'] = [tag_team_parser(i) for i in threewaytag_matches.win_team]
threewaytag_matches['loseteam1'] = [tag_team_parser(i) for i in threewaytag_matches.loseteam1]
threewaytag_matches['loseteam2'] = [tag_team_parser(i) for i in threewaytag_matches.loseteam2]

threewaytag_matches[['win1', 'win2']] = threewaytag_matches.win_team.str.split(" & ",expand=True)
threewaytag_matches[['lose1', 'lose2']] = threewaytag_matches.loseteam1.str.split(" & ",expand=True) 
threewaytag_matches[['lose3', 'lose4']] = threewaytag_matches.loseteam2.str.split(" & ",expand=True) 

threewaytag_matches = threewaytag_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'lose3', 'lose4', 'time']]

threewaytag_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,lose3,lose4,time
366,AEW Dark #76,23.02.2021,SCU (Christopher Daniels & Frankie Kazarian) d...,threewaytag,Christopher Daniels,Frankie Kazarian,Angelico,Jack Evans,Matt Sydal,Mike Sydal,(11:38)
430,AEW Dynamite #69,20.01.2021,The Inner Circle (Chris Jericho & MJF) defeat...,threewaytag,Chris Jericho,MJF,Jake Hager,Sammy Guevara,Ortiz,Santana,(12:38)
607,AEW Dark #64,01.12.2020,The Jurassic Express (Jungle Boy & Luchasaurus...,threewaytag,Jungle Boy,Luchasaurus,Five,Ten,Joey Janela,Sonny Kiss,(9:35)
629,AEW Dark #63,24.11.2020,The Dark Order (Evil Uno & Stu Grayson) defeat...,threewaytag,Evil Uno,Stu Grayson,Luther,Serpentico,Joey Janela,Sonny Kiss,(9:02)
955,AEW Dark #44,28.07.2020,The Inner Circle (Ortiz & Santana) defeat SCU ...,threewaytag,Ortiz,Santana,Christopher Daniels,Frankie Kazarian,Isiah Kassidy,Marq Quen,(11:01)
1346,AEW Full Gear 2019,09.11.2019,SCU (Frankie Kazarian & Scorpio Sky) defeat T...,threewaytag,Frankie Kazarian,Scorpio Sky,Pentagon Jr.,Rey Fenix,Isiah Kassidy,Marq Quen,(13:07)
1413,AEW Fight For The Fallen,13.07.2019,The Dark Order (Evil Uno & Stu Grayson) defeat...,threewaytag,Evil Uno,Stu Grayson,Jungle Boy,Luchasaurus,Angelico,Jack Evans,(15:07)
1425,AEW Fyter Fest - The Buy In,29.06.2019,Best Friends (Chuck Taylor & Trent) defeat SCU...,threewaytag,Chuck Taylor,Trent,Frankie Kazarian,Scorpio Sky,Isiah Kassidy,Marq Quen,(15:56)


#### parsing handicaps

In [377]:
handicap3on1_matches = r[r['matchtype'] == 'handicap3on1']

handicap3on1_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in handicap3on1_matches['result']]
handicap3on1_matches.result = [i.rstrip().lstrip() for i in handicap3on1_matches['result'].str.replace(r"\(w\/.*?\)","")]

handicap3on1_matches['win1'] = [singles_splitter(i)[0] for i in handicap3on1_matches['result']]
handicap3on1_matches['lose_teams'] = [singles_splitter(i)[1] for i in handicap3on1_matches['result']]
handicap3on1_matches[['lose1', 'lose2', 'lose3']] = [re.split(r'[,&()]\s*', i) for i in handicap3on1_matches.lose_teams]

handicap3on1_matches = handicap3on1_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', 'lose3']]


handicap3on1_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3
858,AEW Dynamite #47 - Thursday Night Dynamite,27.08.2020,"Big Swole defeats Dr. Britt Baker DMD, Penelop...",handicap3on1,Big Swole,Dr. Britt Baker DMD,Penelope Ford,Reba


In [378]:
handicap2on1_matches = r[r['matchtype'] == 'handicap2on1']

handicap2on1_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in handicap2on1_matches['result']]
handicap2on1_matches.result = [i.rstrip().lstrip() for i in handicap2on1_matches['result'].str.replace(r"\(w\/.*?\)","")]

handicap2on1_matches['win'] = [singles_splitter(i)[0] for i in handicap2on1_matches['result']]
handicap2on1_matches['win_teams'] = [tag_splitter(i)[0] for i in handicap2on1_matches['result']]
handicap2on1_matches['winteam1'] = [tag_team_parser(i) for i in handicap2on1_matches.win_teams]
handicap2on1_matches[['win1', 'win2']] = handicap2on1_matches.winteam1.str.split(" & ",expand=True) 
handicap2on1_matches['lose_teams'] = [tag_splitter(i)[1] for i in handicap2on1_matches['result']]
handicap2on1_matches['loseteam1'] = [tag_team_parser(i) for i in handicap2on1_matches.lose_teams]
handicap2on1_matches[['lose1', 'lose2']] = handicap2on1_matches.loseteam1.str.split(" & ",expand=True) 
handicap2on1_matches['lose1'] = [i[2:] if i[:2] == 's ' else i for i in handicap2on1_matches.lose1]

handicap2on1_matches = handicap2on1_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'win2', 'lose1', 'lose2', 'time']]

handicap2on1_matches

,Event,Date,result,matchtype,win1,win2,lose1,lose2,time
903,AEW Dark #47,18.08.2020,Lance Archer defeats Jessy Sorensen & Jon Cru...,handicap2on1,Lance Archer,None,Jessy Sorensen,Jon Cruz,(2:42)
972,AEW Dynamite #40 - Fyter Fest 2020 - Tag 2,08.07.2020,Nyla Rose defeats Kenzie Paige & KiLynn King (...,handicap2on1,Nyla Rose,None,Kenzie Paige,KiLynn King,(2:21)
1192,AEW Dynamite #22,04.03.2020,The Inner Circle (Chris Jericho & Sammy Guevar...,handicap2on1,Chris Jericho,Sammy Guevara,Darby Allin,None,(Unk.)


#### other parses

In [379]:
nineway_matches = r[r['matchtype'] == 'nineway']


nineway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in nineway_matches['result']]
nineway_matches.result = [i.rstrip().lstrip() for i in nineway_matches['result'].str.replace(r"\(w\/.*?\)","")]

nineway_matches['win1'] = [singles_splitter(i)[0] for i in nineway_matches['result']]
nineway_matches['lose'] = [singles_splitter(i)[1] for i in nineway_matches['result']]
nineway_matches[['lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'lose7', 'lose8']] = \
[(multiway_splitter(i)[0], multiway_splitter(i)[1], multiway_splitter(i)[2], multiway_splitter(i)[3], \
  multiway_splitter(i)[4], multiway_splitter(i)[5], multiway_splitter(i)[6], multiway_splitter(i)[7], \
 ) for i in nineway_matches.lose]


nineway_matches = nineway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', \
                                   'lose3', 'lose4', 'lose5', 'lose6', 'lose7', 'lose8', 'time']]


nineway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3,lose4,lose5,lose6,lose7,lose8,time
1062,AEW Double Or Nothing 2020,23.05.2020,Brian Cage defeats Colt Cabana and Darby Alli...,nineway,Brian Cage,Colt Cabana,Darby Allin,Frankie Kazarian,Kip Sabian,Joey Janela,Luchasaurus,Orange Cassidy,Scorpio Sky,(27:28)


In [380]:
sixway_matches = r[r['matchtype'] == 'sixway']


sixway_matches['time'] = [singles_splitter(i)[2] if len(singles_splitter(i)) == 3 else '(Unk.)' for i in sixway_matches['result']]
sixway_matches.result = [i.rstrip().lstrip() for i in sixway_matches['result'].str.replace(r"\(w\/.*?\)","")]

sixway_matches['win1'] = [singles_splitter(i)[0] for i in sixway_matches['result']]
sixway_matches['lose'] = [singles_splitter(i)[1] for i in sixway_matches['result']]
sixway_matches[['lose1', 'lose2', 'lose3', 'lose4', 'lose5']] = \
[(multiway_splitter(i)[0], multiway_splitter(i)[1], multiway_splitter(i)[2], multiway_splitter(i)[3], \
  multiway_splitter(i)[4]) for i in sixway_matches.lose]


sixway_matches = sixway_matches[['Event', 'Date', 'result', 'matchtype', 'win1', 'lose1', 'lose2', \
                                   'lose3', 'lose4', 'lose5', 'time']]


sixway_matches

,Event,Date,result,matchtype,win1,lose1,lose2,lose3,lose4,lose5,time
281,AEW Revolution 2021,07.03.2021,Scorpio Sky defeats Cody Rhodes and Ethan Pag...,sixway,Scorpio Sky,Cody Rhodes,Ethan Page,Lance Archer,Max Caster,Penta El Zero Miedo,(23:07)


### merge

In [406]:
data_frames = [singles_matches, tag_matches, tag6_matches, tag8_matches, tag10_matches, tag12_matches, fourway_matches,\
               threeway_matches, threewaytag_matches, fourwaytag_matches, handicap3on1_matches, handicap2on1_matches,\
               sixway_matches, nineway_matches]

merged_df = pd.concat(data_frames)
merged_df[merged_df.columns] = merged_df.apply(lambda x: x.str.strip())
merged_df['win1'] = [re.sub('\(vakant\)', '', x) for x in merged_df.win1]

merged_df = merged_df[['Event', 'Date', 'result', 'matchtype', 'time',\
                     'win1', 'win2', 'win3', 'win4', 'win5', 'win6',\
                     'lose1', 'lose2', 'lose3', 'lose4', 'lose5', 'lose6', 'lose7', 'lose8']]

draws = pd.read_csv('all_draws.csv')
merged_df = pd.concat([merged_df, draws])

merged_df.to_csv('merged_' + today + '.csv')

In [408]:
merged_df

,Event,Date,result,matchtype,time,win1,win2,win3,win4,win5,win6,lose1,lose2,lose3,lose4,lose5,lose6,lose7,lose8
1,AEW Dynamite #84 - Blood & Guts,05.05.2021,Cody Rhodes (w/Arn Anderson) defeats QT Marsha...,singles,(11:45),Cody Rhodes,NaN,NaN,NaN,NaN,NaN,QT Marshall,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AEW Dynamite #84 - Blood & Guts,05.05.2021,Dr. Britt Baker DMD (w/Rebel) defeats Julia Ha...,singles,(1:30),Dr. Britt Baker DMD,NaN,NaN,NaN,NaN,NaN,Julia Hart,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AEW Dark: Elevation #8,03.05.2021,Miro defeats Will Allday (4:32),singles,(4:32),Miro,NaN,NaN,NaN,NaN,NaN,Will Allday,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AEW Dark: Elevation #8,03.05.2021,Abadon defeats Ryo Mizunami (6:51),singles,(6:51),Abadon,NaN,NaN,NaN,NaN,NaN,Ryo Mizunami,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AEW Dark: Elevation #8,03.05.2021,Jon Moxley defeats Andrew Palace by referee's ...,singles,(3:08),Jon Moxley,NaN,NaN,NaN,NaN,NaN,Andrew Palace,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,AEW Fyter Fest,29.06.2019,Cody (w/Brandi Rhodes) vs. Darby Allin - Time ...,draw,(20:00),Cody,NaN,NaN,NaN,NaN,NaN,Darby Allin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AEW Dark #56,13.10.2020,Brandon Cutler vs. Peter Avalon - Double DQ,draw,(7:27),Brandon Cutler,NaN,NaN,NaN,NaN,NaN,Peter Avalon,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AEW Dark #52,15.09.2020,Brandon Cutler vs. Peter Avalon - Double Count...,draw,(6:45),Brandon Cutler,NaN,NaN,NaN,NaN,NaN,Peter Avalon,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AEW Dynamite #4,23.10.2019,Jon Moxley vs. PAC - Time Limit Draw (12:10),draw,(12:10),Jon Moxley,NaN,NaN,NaN,NaN,NaN,PAC,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Trueskill

In [409]:
alias_dict = pd.read_excel('alias_dict.xlsx')
aliases = list(alias_dict.alias)

def alias_collapser(name):
    y = list(alias_dict[alias_dict['alias'] == name]['name'])
    return y[0]

In [410]:
all_matches = pd.read_csv('merged_' + today + '.csv')
all_matches = all_matches.drop(columns=['Unnamed: 0'])
all_matches['Date'] = pd.to_datetime(all_matches.Date, format='%d.%m.%Y')
all_matches = all_matches.sort_values('Date')


all_matches.tail()

,Event,Date,result,matchtype,time,win1,win2,win3,win4,win5,win6,lose1,lose2,lose3,lose4,lose5,lose6,lose7,lose8
1411,AEW Dynamite #84 - Blood & Guts,2021-05-05,SCU (Christopher Daniels & Frankie Kazarian) d...,fourwaytag,(9:12),Christopher Daniels,Frankie Kazarian,NaN,NaN,NaN,NaN,Brian Pillman Jr.,Griff Garrison,Jungle Boy,Luchasaurus,Anthony Bowens,Max Caster,NaN,NaN
833,AEW Dynamite #84 - Blood & Guts,2021-05-05,Eddie Kingston & Jon Moxley defeat Kenny Omega...,tag,(8:03),Eddie Kingston,Jon Moxley,NaN,NaN,NaN,NaN,Kenny Omega,MT Nakazawa,NaN,NaN,NaN,NaN,NaN,NaN
1379,AEW Dynamite #84 - Blood & Guts,2021-05-05,"The Pinnacle (Cash Wheeler, Dax Harwood, MJF, ...",tag10,(17:00),Cash Wheeler,Dax Harwood,MJF,Shawn Spears,Wardlow,NaN,Chris Jericho,Jake Hager,Ortiz,Sammy Guevara,Santana,NaN,NaN,NaN
1,AEW Dynamite #84 - Blood & Guts,2021-05-05,Dr. Britt Baker DMD (w/Rebel) defeats Julia Ha...,singles,(1:30),Dr. Britt Baker DMD,NaN,NaN,NaN,NaN,NaN,Julia Hart,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,AEW Dynamite #84 - Blood & Guts,2021-05-05,Cody Rhodes (w/Arn Anderson) defeats QT Marsha...,singles,(11:45),Cody Rhodes,NaN,NaN,NaN,NaN,NaN,QT Marshall,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [411]:
x = Rating(25, 8)
x.mu

Rating()

trueskill.Rating(mu=25.000, sigma=8.333)

In [414]:
def create_player_frame():
    '''
    initializes dataframe
    '''
    player_frame = pd.DataFrame(columns=['name', 'mu', 'sigma'])
    return player_frame

def create_macro_player_frame():
    '''
    initializes by-date dataframe
    '''
    player_frame = pd.DataFrame(columns=['name', 'mu', 'sigma','Date', 'Result'])
    return player_frame


def player_add(name, player_frame):
    '''
    adds player to created player frame
    '''
    x = Rating()
    player_frame  = player_frame.append({'name': name, \
                                         'mu': x.mu, 'sigma': x.sigma}, ignore_index=True)
    
    return player_frame

def player_array(name, player_frame):
    '''
    returns mu, sigma (in that order) of player's trueskill
    '''
    
    mu = player_frame[player_frame['name'] == name]['mu']
    sigma = player_frame[player_frame['name'] == name]['sigma']
    
    mu = list(mu)
    sigma = list(sigma)
    
    mu1 = mu[0]
    sigma1 = sigma[0]
    
    return [mu1, sigma1]

def victory_1v1(winner, loser, player_frame):
    '''
    updates player_frame after a 1v1 matchup that is not a tie
    '''
    
    winner_mu, winner_sigma = player_array(winner, player_frame)
    loser_mu, loser_sigma = player_array(loser, player_frame)
    
    win_rate = Rating(winner_mu, winner_sigma)
    lose_rate = Rating(loser_mu, loser_sigma)
    
    result = rate([(win_rate,), (lose_rate,)], ranks=[0,1])
    (win,), (lose,), = result
    
    player_frame = player_frame[player_frame['name'] != winner]
    player_frame = player_frame[player_frame['name'] != loser]
    
    player_frame  = player_frame.append({'name': loser, \
                                         'mu': lose.mu, 'sigma': lose.sigma}, ignore_index=True)
    player_frame  = player_frame.append({'name': winner, \
                                         'mu': win.mu, 'sigma': win.sigma}, ignore_index=True)
    
    return player_frame

def get_rating(name, player_frame):
    '''
    '''
    mu, sigma = player_array(name, player_frame)
    rating = Rating(mu, sigma)
    return rating

def victory_singles(result_list, player_frame):
    '''
    updates player_frame after any singles matchup that is not a tie (regardless of participant count)
    '''
    rate_pt1 = []
    for item in result_list:
        x = (get_rating(item, player_frame),)
        rate_pt1.append(x)
        
    y = len(result_list)
    rate_pt2 = [0]
    for i in [1] * (y-1):
        rate_pt2.append(1)
        
    result = rate(rate_pt1, ranks=rate_pt2)
    
    player_frame_old = player_frame
    
    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame = player_frame.append({'name': result_list[i], \
                                         'mu': result[i][0].mu, 'sigma': result[i][0].sigma}, ignore_index=True)
        
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[1:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
    return player_frame

def victory_draw(result_list, player_frame):
    '''
    updates player_frame after any singles matchup that is a tie (regardless of participant count)
    '''
    rate_pt1 = []
    for item in result_list:
        x = (get_rating(item, player_frame),)
        rate_pt1.append(x)
        
    y = len(result_list)
    rate_pt2 = [0]
    for i in [1] * (y-1):
        rate_pt2.append(0)
        
    result = rate(rate_pt1, ranks=[0,0])
    
    player_frame_old = player_frame
    
    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame = player_frame.append({'name': result_list[i], \
                                         'mu': result[i][0].mu, 'sigma': result[i][0].sigma}, ignore_index=True)
        
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[0:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
    return player_frame

def victory_tag(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)

    y1 = Rating(get_rating(result_list[2], player_frame),)
    y2 = Rating(get_rating(result_list[3], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[2:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_threewaytag(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)

    y1 = Rating(get_rating(result_list[2], player_frame),)
    y2 = Rating(get_rating(result_list[3], player_frame),)
    
    z1 = Rating(get_rating(result_list[4], player_frame),)
    z2 = Rating(get_rating(result_list[5], player_frame),)

    rate_pt2 = [0, 1, 1]
    
    result = rate([(x1,x2), (y1,y2), (z1, z2)], ranks=[0,1,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[2:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_fourwaytag(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)

    y1 = Rating(get_rating(result_list[2], player_frame),)
    y2 = Rating(get_rating(result_list[3], player_frame),)
    
    z1 = Rating(get_rating(result_list[4], player_frame),)
    z2 = Rating(get_rating(result_list[5], player_frame),)
    
    v1 = Rating(get_rating(result_list[6], player_frame),)
    v2 = Rating(get_rating(result_list[7], player_frame),)

    rate_pt2 = [0, 1, 1, 1]
    
    result = rate([(x1,x2), (y1,y2), (z1, z2), (v1, v2)], ranks=[0,1,1,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[2:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag6(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)

    y1 = Rating(get_rating(result_list[3], player_frame),)
    y2 = Rating(get_rating(result_list[4], player_frame),)
    y3 = Rating(get_rating(result_list[5], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2,x3), (y1,y2,y3)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[3:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag8(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)
    x4 = Rating(get_rating(result_list[3], player_frame),)

    y1 = Rating(get_rating(result_list[4], player_frame),)
    y2 = Rating(get_rating(result_list[5], player_frame),)
    y3 = Rating(get_rating(result_list[6], player_frame),)
    y4 = Rating(get_rating(result_list[7], player_frame),)

    rate_pt2 = [0, 1]

    
    result = rate([(x1,x2,x3,x4), (y1,y2,y3,y4)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[4:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag10(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)
    x4 = Rating(get_rating(result_list[3], player_frame),)
    x5 = Rating(get_rating(result_list[4], player_frame),)

    y1 = Rating(get_rating(result_list[5], player_frame),)
    y2 = Rating(get_rating(result_list[6], player_frame),)
    y3 = Rating(get_rating(result_list[7], player_frame),)
    y4 = Rating(get_rating(result_list[8], player_frame),)
    y5 = Rating(get_rating(result_list[9], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2,x3,x4,x5), (y1,y2,y3,y4,y5)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[5:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_tag12(result_list, player_frame):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    x1 = Rating(get_rating(result_list[0], player_frame),)
    x2 = Rating(get_rating(result_list[1], player_frame),)
    x3 = Rating(get_rating(result_list[2], player_frame),)
    x4 = Rating(get_rating(result_list[3], player_frame),)
    x5 = Rating(get_rating(result_list[4], player_frame),)
    x6 = Rating(get_rating(result_list[5], player_frame),)

    y1 = Rating(get_rating(result_list[6], player_frame),)
    y2 = Rating(get_rating(result_list[7], player_frame),)
    y3 = Rating(get_rating(result_list[8], player_frame),)
    y4 = Rating(get_rating(result_list[9], player_frame),)
    y5 = Rating(get_rating(result_list[10], player_frame),)
    y6 = Rating(get_rating(result_list[11], player_frame),)

    rate_pt2 = [0, 1]
    
    result = rate([(x1,x2,x3,x4,x5,x6), (y1,y2,y3,y4,y5,y6)], ranks=[0,1])

    result = [item for sublist in result for item in sublist]

    y = len(result_list)
    
    player_frame_old = player_frame

    for item in result_list:
        player_frame = player_frame[player_frame['name'] != item]
    
    for i in range(0,y):
        player_frame  = player_frame.append({'name': result_list[i], \
                                         'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
    #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
    for item in result_list[6:]:
        if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
            player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_handicap3on1(result_list, player_frame, date):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    if date == pd.to_datetime('2020-08-27').normalize():
        
        x1 = Rating(get_rating('Big Swole', player_frame),)
        x2 = Rating(25,3)
        x3 = Rating(25,3)

        y1 = Rating(get_rating('Dr. Britt Baker DMD', player_frame),)
        y2 = Rating(get_rating('Penelope Ford', player_frame),)
        y3 = Rating(get_rating('Reba', player_frame),)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2,x3), (y1,y2,y3)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

def victory_handicap2on1(result_list, player_frame, date):
    '''
    updates player_frame after a tag matchup that is not a tie
    '''
    
    if date == pd.to_datetime('2020-08-12').normalize():
        
        x1 = Rating(get_rating('Lance Archer', player_frame),)
        x2 = Rating(25,1)

        y1 = Rating(get_rating('Jessy Sorensen', player_frame),)
        y2 = Rating(get_rating('Jon Cruz', player_frame),)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
                
    if date == pd.to_datetime('2020-07-02').normalize():
        
        x1 = Rating(get_rating('Nyla Rose', player_frame),)
        x2 = Rating(25,1)

        y1 = Rating(get_rating('Kenzie Paige', player_frame),)
        y2 = Rating(get_rating('KiLynn King', player_frame),)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu
                
    if date == pd.to_datetime('2020-03-04').normalize():
        
        x1 = Rating(get_rating('Chris Jericho', player_frame),)
        x2 = Rating(get_rating('Sammy Guevara', player_frame),)
        
        y1 = Rating(get_rating('Darby Allin', player_frame),)
        y2 = Rating(25,1)
        
        print(x1)

        rate_pt2 = [0, 1]
    
        result = rate([(x1,x2), (y1,y2)], ranks=[0,1])

        result = [item for sublist in result for item in sublist]

        y = len(result_list)
    
        player_frame_old = player_frame

        for item in result_list:
            player_frame = player_frame[player_frame['name'] != item]

        for i in range(0,y):
            player_frame  = player_frame.append({'name': result_list[i], \
                                             'mu': result[i].mu, 'sigma': result[i].sigma}, ignore_index=True)
    
        #added 4.4.2021: should prevent losers in multiway matches from upping their rankings because of competition
        for item in result_list[2:]:
            if get_rating(item, player_frame).mu > get_rating(item, player_frame_old).mu:
                player_frame.loc[(player_frame.name == item),'mu'] = get_rating(item, player_frame_old).mu

    return player_frame

### does the deal

For every day since May 1, 2019, checks if there was at least one AEW match that happened that day in the data. If there was, adds each match to the larger results frame. Adds player to the results frame if they are new. If the day is December 31, performs rankings reset.

In [415]:
player_frame = create_player_frame()
macro_player_frame = create_macro_player_frame()

pandas_today = pd.to_datetime('today').normalize()
x = pd.to_datetime('2019-05-01').normalize()

while x < pandas_today:
    print(x)
    
    #optionally can add (all_matches['matchtype'] == 'singles') for specific divisions only
    day_matches = all_matches[(all_matches['Date'] == x)]
    day_results = [item for item in day_matches.result]

    for index, row in day_matches.iterrows():
        placement = [item for item in [row.win1, row.win2, row.win3, row.win4, row.win5, row.win6,\
                                       row.lose1, row.lose2, row.lose3, row.lose4, row.lose5, row.lose6,\
                                      row.lose7, row.lose8] if str(item) != 'nan']
        placement = [alias_collapser(item) if item in aliases else item for item in placement]

        for item in placement:
            if item not in set(player_frame.name):
                player_frame = player_add(item, player_frame)
        if row.matchtype in ['singles', 'threeway', 'fourway', 'sixway', 'nineway']:
            player_frame = victory_singles(placement, player_frame)
        if row.matchtype in ['draw']:
            player_frame = victory_draw(placement, player_frame)
        if row.matchtype in ['tag']:
            player_frame = victory_tag(placement, player_frame)
        if row.matchtype in ['tag6']:
            player_frame = victory_tag6(placement, player_frame)
        if row.matchtype in ['tag8']:
            player_frame = victory_tag8(placement, player_frame)
        if row.matchtype in ['tag10']:
            player_frame = victory_tag10(placement, player_frame)
        if row.matchtype in ['tag12']:
            player_frame = victory_tag12(placement, player_frame)
        if row.matchtype in ['threewaytag']:
            player_frame = victory_threewaytag(placement, player_frame)
        if row.matchtype in ['fourwaytag']:
            player_frame = victory_fourwaytag(placement, player_frame)
        if row.matchtype in ['handicap3on1']:
            player_frame = victory_handicap3on1(placement, player_frame,x)
        if row.matchtype in ['handicap2on1']:
            player_frame = victory_handicap2on1(placement, player_frame,x)
    
    intermediary = player_frame
    intermediary['Date'] = x
    intermediary['Result'] = ''
    
    for index, row in intermediary.iterrows():
        for item in day_results:
            if str(row['name']) in item:
                intermediary.loc[index, 'Result'] = item
                
    macro_player_frame = pd.concat([macro_player_frame, intermediary])
    
    if (x.month == 12) & (x.day == 31):    
        player_frame = player_frame[~(player_frame.sigma == 8.33)]
        player_frame.mu = [25 + ((i - 25) / 2) for i in player_frame.mu]
        player_frame.sigma = [8.33 for i in player_frame.sigma]
    
    x = x + pd.DateOffset(1)

2019-05-01 00:00:00
2019-05-02 00:00:00
2019-05-03 00:00:00
2019-05-04 00:00:00
2019-05-05 00:00:00
2019-05-06 00:00:00
2019-05-07 00:00:00
2019-05-08 00:00:00
2019-05-09 00:00:00
2019-05-10 00:00:00
2019-05-11 00:00:00
2019-05-12 00:00:00
2019-05-13 00:00:00
2019-05-14 00:00:00
2019-05-15 00:00:00
2019-05-16 00:00:00
2019-05-17 00:00:00
2019-05-18 00:00:00
2019-05-19 00:00:00
2019-05-20 00:00:00
2019-05-21 00:00:00
2019-05-22 00:00:00
2019-05-23 00:00:00
2019-05-24 00:00:00
2019-05-25 00:00:00
2019-05-26 00:00:00
2019-05-27 00:00:00
2019-05-28 00:00:00
2019-05-29 00:00:00
2019-05-30 00:00:00
2019-05-31 00:00:00
2019-06-01 00:00:00
2019-06-02 00:00:00
2019-06-03 00:00:00
2019-06-04 00:00:00
2019-06-05 00:00:00
2019-06-06 00:00:00
2019-06-07 00:00:00
2019-06-08 00:00:00
2019-06-09 00:00:00
2019-06-10 00:00:00
2019-06-11 00:00:00
2019-06-12 00:00:00
2019-06-13 00:00:00
2019-06-14 00:00:00
2019-06-15 00:00:00
2019-06-16 00:00:00
2019-06-17 00:00:00
2019-06-18 00:00:00
2019-06-19 00:00:00


2020-06-16 00:00:00
2020-06-17 00:00:00
2020-06-18 00:00:00
2020-06-19 00:00:00
2020-06-20 00:00:00
2020-06-21 00:00:00
2020-06-22 00:00:00
2020-06-23 00:00:00
2020-06-24 00:00:00
2020-06-25 00:00:00
2020-06-26 00:00:00
2020-06-27 00:00:00
2020-06-28 00:00:00
2020-06-29 00:00:00
2020-06-30 00:00:00
2020-07-01 00:00:00
2020-07-02 00:00:00
2020-07-03 00:00:00
2020-07-04 00:00:00
2020-07-05 00:00:00
2020-07-06 00:00:00
2020-07-07 00:00:00
2020-07-08 00:00:00
2020-07-09 00:00:00
2020-07-10 00:00:00
2020-07-11 00:00:00
2020-07-12 00:00:00
2020-07-13 00:00:00
2020-07-14 00:00:00
2020-07-15 00:00:00
2020-07-16 00:00:00
2020-07-17 00:00:00
2020-07-18 00:00:00
2020-07-19 00:00:00
2020-07-20 00:00:00
2020-07-21 00:00:00
2020-07-22 00:00:00
2020-07-23 00:00:00
2020-07-24 00:00:00
2020-07-25 00:00:00
2020-07-26 00:00:00
2020-07-27 00:00:00
2020-07-28 00:00:00
2020-07-29 00:00:00
2020-07-30 00:00:00
2020-07-31 00:00:00
2020-08-01 00:00:00
2020-08-02 00:00:00
2020-08-03 00:00:00
2020-08-04 00:00:00


In [416]:
macro_player_frame = macro_player_frame.groupby(['name', 'mu', 'sigma']).aggregate({"Date": "min", "Result": "max"})

In [417]:
x = macro_player_frame.groupby(by=["name"], dropna=False).count()
matchcounts = pd.DataFrame(x['Date'])

player_frame = player_frame.sort_values('name', ascending=True)

In [418]:
player_frame[player_frame.sigma < 8.333].sort_values('mu', ascending=False).to_csv('ranking_output_singles_' + today + '.csv', index=False)


In [419]:
player_frame[player_frame.sigma < 8.333].sort_values('mu', ascending=False).to_csv('ranking_output_' + today + '.csv', index=False)
macro_player_frame.sort_values('mu', ascending=False).to_csv('macro_output_' + today + '.csv')

macro_player_frame.sort_values('mu', ascending=False).to_csv('macro_output.csv')

In [420]:
macro_player_frame.sort_values('Date').filter(like = 'Nyla Rose', axis=0)

Date  \
name      mu        sigma                 
Nyla Rose 23.193658 5.429410 2019-05-25   
          21.995731 4.577329 2019-06-29   
          21.194612 4.339793 2019-10-02   
          22.678838 4.083843 2019-10-22   
          24.038802 3.983094 2019-11-08   
          25.341580 3.803664 2019-11-13   
          25.937549 3.646443 2019-12-04   
          25.423307 3.498637 2019-12-31   
          24.445156 5.464429 2020-01-01   
          26.302210 5.081015 2020-01-28   
          28.623827 4.660875 2020-01-29   
          32.229695 4.176253 2020-02-12   
          33.656966 3.903861 2020-02-29   
          34.670699 3.800195 2020-03-11   
          35.147030 3.701813 2020-05-06   
          36.132677 3.556901 2020-05-20   
          34.252207 3.252903 2020-05-23   
          34.612970 3.152528 2020-06-03   
          35.462735 3.051339 2020-06-10   
          34.455654 3.004849 2020-08-03   
          34.467342 3.000066 2020-08-25   
          34.905518 2.925189 2020-09-09   
          34.974328 2.903915 2020-09-29   
          34.979707 2.902150 2020-10-13   
          35.001957 2.892903 2020-11-04   
          34.084460 2.733719 2020-11-07   
          34.118866 2.722392 2020-12-08   
          34.125112 2.720523 2020-12-22   
          29.562556 8.330000 2021-01-01   
          32.095302 7.453907 2021-01-05   
          33.747874 6.867458 2021-02-09   
          36.860957 5.910200 2021-02-22   
          39.771904 5.185592 2021-02-24   
          41.140465 4.766503 2021-03-01   
          38.151635 4.278690 2021-03-03   
          37.646528 4.205254 2021-03-10   
          34.785060 3.771441 2021-03-24   
          36.533622 3.650045 2021-03-31   
          36.284805 3.621716 2021-04-09   
          36.368558 3.588009 2021-04-13   
          35.962816 3.514200 2021-04-19   
          35.564653 3.451364 2021-04-26   
          35.678574 3.405286 2021-05-03   

                                                                         Result  
name      mu        sigma                                                        
Nyla Rose 23.193658 5.429410  Dr. Britt Baker DMD defeats Awesome Kong  and ...  
          21.995731 4.577329   Riho defeats Nyla Rose and Yuka Sakazaki (13:02)  
          21.194612 4.339793             (vakant)Riho defeats Nyla Rose (13:26)  
          22.678838 4.083843  Nyla Rose defeats Leva Bates (w/Peter Avalon) ...  
          24.038802 3.983094  Leva Bates & Nyla Rose defeat Shalandra Royal ...  
          25.341580 3.803664               Nyla Rose defeats Dani Jordyn (1:34)  
          25.937549 3.646443  Nyla Rose defeats Leva Bates (w/Peter Avalon) ...  
          25.423307 3.498637                     (vakant)Riho defeats Nyla Rose  
          24.445156 5.464429  Riho  defeats Dr. Britt Baker DMD and Hikaru S...  
          26.302210 5.081015                    Nyla Rose defeats Shanna (8:05)  
          28.623827 4.660875                 Nyla Rose defeats Big Swole (8:44)  
          32.229695 4.176253                    Nyla Rose defeats Riho  (13:07)  
          33.656966 3.903861         Nyla Rose  defeats Kris Statlander (12:49)  
          34.670699 3.800195  Bea Priestley & Nyla Rose defeat Hikaru Shida ...  
          35.147030 3.701813              Nyla Rose defeats Kenzie Paige (2:19)  
          36.132677 3.556901  Dr. Britt Baker DMD & Nyla Rose defeat Hikaru ...  
          34.252207 3.252903            Hikaru Shida defeats Nyla Rose  (16:34)  
          34.612970 3.152528                 Nyla Rose defeats Big Swole (6:33)  
          35.462735 3.051339  Nyla Rose defeats Kenzie Paige & KiLynn King (...  
          34.455654 3.004849  Anna Jay & Tay Conti defeat Ariane Andrew & Ny...  
          34.467342 3.000066  Nyla Rose (w/Vickie Guerrero) defeats KiLynn K...  
          34.905518 2.925189  Nyla Rose (w/Vickie Guerrero) defeats Tay Cont...  
          34.974328 2.903915  Nyla Rose (w/Vickie Guerrero) defeats Rache Ch...  
          34.979707 2.902150  Nyla Rose (w/Vickie

## Double-macro frame (all date comparison)

In [421]:
x = pd.read_csv('macro_output.csv')
x = x.sort_values(['name', 'Date'])

x.Date

3148    2021-01-19
2745    2021-02-23
2925    2021-03-02
3024    2021-03-09
3259    2021-03-29
           ...    
4051    2020-09-04
3383    2021-01-01
3540    2021-01-12
4108    2021-01-19
4195    2021-04-12
Name: Date, Length: 4690, dtype: object

In [422]:
x = pd.read_csv('macro_output.csv')
x = x.sort_values(['name', 'Date'])

double_macro = pd.DataFrame(columns=['name', 'mu', 'sigma', 'Date'])

zz = 0
print(len(set(x.name)))

for guy in set(x.name):
    print(zz)
    print(guy)
    y = x[x.name == guy]
    y = y.reset_index()
    y = y[['name', 'mu', 'sigma', 'Date']]
    y['Date'] = pd.to_datetime(y.Date, format='%Y-%m-%d')
    
    date1 = y.Date[0] #first appearance
    date2 = y.Date[len(y) - 1] #last appearance
    
    cutoff = date2 + relativedelta(months=+3)
    
    print(date1)
    print(cutoff)
    
    check_date = date1
    most_recent_match = date1
    
    while (check_date < cutoff) & (check_date < pandas_today):
        all_dates = list(y.Date)
        
        if (check_date not in all_dates) & (check_date < most_recent_match + pd.DateOffset(90)):
            y = y.append({'name': guy, 'mu': None, 'sigma': None, 'Date': check_date}, ignore_index=True)
            
        if (check_date in all_dates) & (check_date != '2021-01-01 00:00:00'):
            most_recent_match = check_date
            
        check_date = check_date + pd.DateOffset(1)
        
    y = y.sort_values('Date')
    y = y.fillna(method='ffill')
    
    double_macro = pd.concat([double_macro, y])
    
    zz = zz + 1

349
0
Freya States
2020-12-08 00:00:00
2021-04-01 00:00:00
1
Azrieal
2021-03-06 00:00:00
2021-06-09 00:00:00
2
QT Marshall
2019-10-29 00:00:00
2021-08-05 00:00:00
3
Terrell Hughes
2020-11-10 00:00:00
2021-08-04 00:00:00
4
El Lindaman
2019-05-25 00:00:00
2021-04-01 00:00:00
5
Ganon Jones
2021-04-27 00:00:00
2021-08-03 00:00:00
6
Cody Rhodes
2019-05-25 00:00:00
2021-08-05 00:00:00
7
Reka Tehaka
2021-04-09 00:00:00
2021-08-03 00:00:00
8
Alex Chamberlain
2020-08-11 00:00:00
2021-04-01 00:00:00
9
El Australiano
2021-01-12 00:00:00
2021-04-19 00:00:00
10
Diamante
2020-01-21 00:00:00
2021-08-04 00:00:00
11
Stone Rockwell
2021-04-13 00:00:00
2021-07-19 00:00:00
12
Lady Frost
2020-11-24 00:00:00
2021-04-01 00:00:00
13
Griff Garrison
2020-06-16 00:00:00
2021-08-05 00:00:00
14
Aaron Solow
2020-07-21 00:00:00
2021-07-28 00:00:00
15
Serpentico
2020-05-19 00:00:00
2021-08-04 00:00:00
16
Queen Aminata
2021-04-13 00:00:00
2021-07-20 00:00:00
17
KENTA
2021-02-10 00:00:00
2021-05-10 00:00:00
18
KTB
2020

149
Marty Casaus
2021-04-27 00:00:00
2021-07-27 00:00:00
150
Big Game Leroy
2020-06-02 00:00:00
2021-04-01 00:00:00
151
Bobby Wayward
2020-12-01 00:00:00
2021-04-01 00:00:00
152
Matt Hardy
2020-05-06 00:00:00
2021-08-03 00:00:00
153
Ricky Starks
2020-06-17 00:00:00
2021-08-04 00:00:00
154
JJ Garrett
2021-02-23 00:00:00
2021-06-02 00:00:00
155
Shaq
2021-03-03 00:00:00
2021-06-03 00:00:00
156
Spencer Slade
2021-04-13 00:00:00
2021-08-04 00:00:00
157
Steff MK
2021-04-20 00:00:00
2021-07-20 00:00:00
158
Alex Reynolds
2019-10-16 00:00:00
2021-08-04 00:00:00
159
Joe Alonzo
2020-03-24 00:00:00
2021-04-01 00:00:00
160
Maxx Stardom
2020-09-15 00:00:00
2021-04-01 00:00:00
161
Jazmin Allure
2020-12-22 00:00:00
2021-07-06 00:00:00
162
James Tapia
2020-12-29 00:00:00
2021-04-01 00:00:00
163
George South Jr.
2020-12-01 00:00:00
2021-04-01 00:00:00
164
Simon Lotto
2020-04-06 00:00:00
2021-04-01 00:00:00
165
Ivelisse
2020-07-22 00:00:00
2021-06-02 00:00:00
166
Lindsay Snow
2020-11-17 00:00:00
2021-04-

295
Tony Vincita
2020-10-27 00:00:00
2021-05-09 00:00:00
296
Wardlow
2020-02-19 00:00:00
2021-08-05 00:00:00
297
Awesome Kong
2019-05-25 00:00:00
2021-04-01 00:00:00
298
Ariel Dominguez
2020-11-06 00:00:00
2021-04-01 00:00:00
299
Liam Gray
2021-04-26 00:00:00
2021-08-04 00:00:00
300
Shazza McKenzie
2019-11-08 00:00:00
2020-04-01 00:00:00
301
Darius Martin
2020-10-27 00:00:00
2021-06-02 00:00:00
302
AJ Kirsch
2021-01-19 00:00:00
2021-04-19 00:00:00
303
Jimmy Havoc
2019-06-29 00:00:00
2021-04-01 00:00:00
304
Powerhouse Hobbs
2020-07-07 00:00:00
2021-08-04 00:00:00
305
Joey O'Riley
2020-11-17 00:00:00
2021-04-01 00:00:00
306
Thunder Rosa
2020-09-02 00:00:00
2021-08-03 00:00:00
307
Orange Cassidy
2019-10-30 00:00:00
2021-08-03 00:00:00
308
Katalina Perez
2020-10-27 00:00:00
2021-07-12 00:00:00
309
Alex Gracia
2020-09-29 00:00:00
2021-07-05 00:00:00
310
Brian Pillman Jr.
2020-07-07 00:00:00
2021-08-05 00:00:00
311
Kenzie Paige
2020-05-06 00:00:00
2021-04-01 00:00:00
312
Tony Vega
2021-02-23

In [423]:
double_macro

,name,mu,sigma,Date
0,Freya States,16.858482,6.253237,2020-12-08
3,Freya States,16.858482,6.253237,2020-12-09
4,Freya States,16.858482,6.253237,2020-12-10
5,Freya States,16.858482,6.253237,2020-12-11
6,Freya States,16.858482,6.253237,2020-12-12
...,...,...,...,...
3,Kit Sackett,18.767749,6.536141,2021-05-04
22,Kit Sackett,18.767749,6.536141,2021-05-05
23,Kit Sackett,18.767749,6.536141,2021-05-06
24,Kit Sackett,18.767749,6.536141,2021-05-07


In [424]:
double_macro.sort_values('name', ascending=False).to_csv('double_macro_' + today + '.csv')
double_macro.sort_values('name', ascending=False).to_csv('double_macro.csv')

In [425]:
double_macro[double_macro.Date == '2020-10-01'].sort_values('mu', ascending=False)

,name,mu,sigma,Date
375,Jon Moxley,45.259259,3.341486,2020-10-01
192,Brodie Lee,39.454657,3.391287,2020-10-01
514,Cody Rhodes,36.628468,2.629880,2020-10-01
512,Kenny Omega,36.241249,3.516621,2020-10-01
198,Lance Archer,36.232059,3.513808,2020-10-01
...,...,...,...,...
151,KiLynn King,11.053681,4.257432,2020-10-01
456,Peter Avalon,9.459326,3.711526,2020-10-01
201,Lee Johnson,9.209689,3.867241,2020-10-01
201,Shawn Dean,7.343074,3.898970,2020-10-01


In [426]:
date1 = pd.to_datetime('2019-05-26').normalize()

while date1 < pandas_today:
    z = double_macro[double_macro.Date == date1]
    z = z.sort_values('mu', ascending=False)
    z = z.reset_index()
    bottom = len(z)
    
    print(date1, '|', z.name[0], '|', z.name[bottom - 1])
    
    date1 = date1 + pd.DateOffset(1)
    


2019-05-26 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-05-27 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-05-28 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-05-29 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-05-30 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-05-31 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-01 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-02 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-03 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-04 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-05 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-06 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-07 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-08 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-09 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-10 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-11 00:00:00 | Dr. Britt Baker DMD | Kenny Omega
2019-06-12 00:00:00 | Dr. Britt Baker DMD | Kenn

2020-01-09 00:00:00 | Riho | Brandon Cutler
2020-01-10 00:00:00 | Riho | Brandon Cutler
2020-01-11 00:00:00 | Riho | Brandon Cutler
2020-01-12 00:00:00 | Riho | Brandon Cutler
2020-01-13 00:00:00 | Riho | Brandon Cutler
2020-01-14 00:00:00 | Riho | Brandon Cutler
2020-01-15 00:00:00 | Riho | Brandon Cutler
2020-01-16 00:00:00 | Riho | Brandon Cutler
2020-01-17 00:00:00 | Riho | Brandon Cutler
2020-01-18 00:00:00 | Riho | Brandon Cutler
2020-01-19 00:00:00 | Riho | Brandon Cutler
2020-01-20 00:00:00 | Riho | Brandon Cutler
2020-01-21 00:00:00 | Riho | Brandon Cutler
2020-01-22 00:00:00 | Kenny Omega | Brandon Cutler
2020-01-23 00:00:00 | Kenny Omega | Brandon Cutler
2020-01-24 00:00:00 | Kenny Omega | Brandon Cutler
2020-01-25 00:00:00 | Kenny Omega | Brandon Cutler
2020-01-26 00:00:00 | Kenny Omega | Brandon Cutler
2020-01-27 00:00:00 | Kenny Omega | Brandon Cutler
2020-01-28 00:00:00 | Kenny Omega | Brandon Cutler
2020-01-29 00:00:00 | Kenny Omega | Brandon Cutler
2020-01-30 00:00:00 

2020-07-11 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-12 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-13 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-14 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-15 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-16 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-17 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-18 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-19 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-20 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-21 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-22 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-23 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-24 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-25 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-26 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-27 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-28 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-29 00:00:00 | Jon Moxley | Brandon Cutler
2020-07-30 00:00:00 | Jon Moxley | Brandon Cutler


2021-01-03 00:00:00 | Jon Moxley | Shawn Dean
2021-01-04 00:00:00 | Jon Moxley | Shawn Dean
2021-01-05 00:00:00 | Jon Moxley | Fuego Del Sol
2021-01-06 00:00:00 | Kenny Omega | Fuego Del Sol
2021-01-07 00:00:00 | Kenny Omega | Fuego Del Sol
2021-01-08 00:00:00 | Kenny Omega | Fuego Del Sol
2021-01-09 00:00:00 | Kenny Omega | Fuego Del Sol
2021-01-10 00:00:00 | Kenny Omega | Fuego Del Sol
2021-01-11 00:00:00 | Kenny Omega | Fuego Del Sol
2021-01-12 00:00:00 | Kenny Omega | Louie Valle
2021-01-13 00:00:00 | Kenny Omega | Louie Valle
2021-01-14 00:00:00 | Kenny Omega | Louie Valle
2021-01-15 00:00:00 | Kenny Omega | Louie Valle
2021-01-16 00:00:00 | Kenny Omega | Louie Valle
2021-01-17 00:00:00 | Kenny Omega | Louie Valle
2021-01-18 00:00:00 | Kenny Omega | Louie Valle
2021-01-19 00:00:00 | Kenny Omega | Fuego Del Sol
2021-01-20 00:00:00 | Kenny Omega | Fuego Del Sol
2021-01-21 00:00:00 | Kenny Omega | Fuego Del Sol
2021-01-22 00:00:00 | Kenny Omega | Fuego Del Sol
2021-01-23 00:00:00 | K